# CSC 480-F25 Lab 5: Games as Adversarial Search (Scrabble Agent)

### Author:
***Grady Schneider***

California Polytechnic State University, San Luis Obispo;
Computer Science & Software Engineering Department

### Overview
This lab covers **Adversarial Search**, beginning with classical Minimax and Alpha-Beta pruning, then transitioning to **Monte Carlo Tree Search (MCTS)** to handle the complexity and imperfect information of Scrabble. We apply the **agentic design approach** (from Lab 2 and the heuristic integration of Lab 4) to construct a sophisticated state evaluation function that guides the MCTS playouts.

---

## Environment Setup and Imports

In [17]:
# Reusing package install from L4
%pip install "autogen-core" "autogen-agentchat" "autogen-ext[openai,azure]"

In [18]:
# @title
%%writefile L5_utils.py
import random
import time
from typing import List, Tuple, Dict, Optional
from itertools import permutations

LETTER_VALUES: Dict[str, int] = {
    "A": 1,
    "B": 3,
    "C": 3,
    "D": 2,
    "E": 1,
    "F": 4,
    "G": 2,
    "H": 4,
    "I": 1,
    "J": 8,
    "K": 5,
    "L": 1,
    "M": 3,
    "N": 1,
    "O": 1,
    "P": 3,
    "Q": 10,
    "R": 1,
    "S": 1,
    "T": 1,
    "U": 1,
    "V": 4,
    "W": 4,
    "X": 8,
    "Y": 4,
    "Z": 10,
    "_": 0,
}
BOARD_SIZE = 15
RACK_SIZE = 7

# Predefined setup for board premiums
# 0=empty, 2=Double Letter, 3=Triple Letter, 4=Double Word, 5=Triple Word
PREMIUM_BOARD = [
    [0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0],
    [2, 0, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 0, 2],
    [0, 0, 4, 0, 0, 0, 2, 0, 2, 0, 0, 0, 4, 0, 0],
    [0, 0, 0, 5, 0, 0, 0, 2, 0, 0, 0, 5, 0, 0, 0],
    [0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0],
    [0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0],
    [0, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0],
    [0, 0, 0, 2, 0, 0, 0, 4, 0, 0, 0, 2, 0, 0, 0],
    [0, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0],
    [0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0],
    [0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0],
    [0, 0, 0, 5, 0, 0, 0, 2, 0, 0, 0, 5, 0, 0, 0],
    [0, 0, 4, 0, 0, 0, 2, 0, 2, 0, 0, 0, 4, 0, 0],
    [2, 0, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 0, 2],
    [0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0],
]

with open("./words.txt") as f:
    DICTIONARY = set(word.strip().upper() for word in f if len(word.strip()) >= 2)


def is_valid_word(word: str) -> bool:
    """Checks if a word is valid."""
    return word.upper() in DICTIONARY


class Move:
    """Represents a potential action taken by a player."""

    def __init__(
        self,
        tiles_placed: List[Tuple[int, int, str]],
        score: int,
        is_pass: bool = False,
    ):
        self.tiles_placed = tiles_placed  # List of (row, col, letter)
        self.score = score
        self.is_pass = is_pass

    def __repr__(self):
        if self.is_pass:
            return "Pass Move"
        # Extract the word formed (simplified, assuming linear placement for representation)
        word = "".join([letter for _, _, letter in self.tiles_placed])
        tile_positions = [(r, c) for r, c, _ in self.tiles_placed]
        return (
            f"Move(Word='{word}', Score={self.score}, Tile Positions={tile_positions})"
        )

    def __eq__(self, value: "Move"):
        # Two Move objects are equal if all fields are equal
        return (
            all(
                tile1 == tile2
                for tile1, tile2 in zip(self.tiles_placed, value.tiles_placed)
            )
            and self.score == value.score
            and self.is_pass == value.is_pass
        )

    def __hash__(self):
        return hash((tuple(self.tiles_placed), self.is_pass))


class ScrabbleState:
    """Represents the current state of the Scrabble game."""

    def __init__(
        self,
        board: List[List[str]],
        tile_pool: List[str],
        racks: Dict[int, List[str]],
        current_player: int,
        scores: Dict[int, int],
    ):
        self.board = board
        self.tile_pool = tile_pool
        self.racks = racks
        self.current_player = current_player
        self.scores = scores
        self.passes_in_a_row = 0

    def __repr__(self) -> str:
        """Return a readable representation of the Scrabble game state."""

        # Helper function to display the board
        def format_board():
            lines = []
            lines.append("    " + " ".join(f"{i:2d}" for i in range(BOARD_SIZE)))
            lines.append("   +" + "---" * BOARD_SIZE + "+")
            for r in range(BOARD_SIZE):
                row_str = f"{r:2d} |"
                for c in range(BOARD_SIZE):
                    cell = self.board[r][c]
                    if cell == "":
                        # Show premium squares on empty cells
                        premium = PREMIUM_BOARD[r][c]
                        if premium == 2:
                            row_str += " ² "  # Double letter
                        elif premium == 3:
                            row_str += " ³ "  # Triple letter
                        elif premium == 4:
                            row_str += " * "  # Double word
                        elif premium == 5:
                            row_str += " # "  # Triple word
                        else:
                            row_str += " · "  # Empty
                    else:
                        row_str += f" {cell} "
                row_str += "|"
                lines.append(row_str)
            lines.append("   +" + "---" * BOARD_SIZE + "+")
            return "\n".join(lines)

        # Build the representation
        output = []
        output.append("=" * 60)
        output.append("SCRABBLE GAME STATE")
        output.append("=" * 60)

        # Current player and scores
        output.append(f"\nCurrent Player: Player {self.current_player}")
        output.append(f"Scores: Player 1: {self.scores[1]}, Player 2: {self.scores[2]}")
        output.append(f"Consecutive Passes: {self.passes_in_a_row}")

        # Player racks
        output.append(
            f"\nPlayer 1 Rack: {' '.join(self.racks[1])} ({len(self.racks[1])} tiles)"
        )
        output.append(
            f"Player 2 Rack: {' '.join(self.racks[2])} ({len(self.racks[2])} tiles)"
        )

        # Tile pool
        output.append(f"\nTiles Remaining in Pool: {len(self.tile_pool)}")

        # Board
        output.append("\nBoard:")
        output.append(
            "Legend: · = empty, ² = 2x letter, ³ = 3x letter, * = 2x word, # = 3x word"
        )
        output.append(format_board())

        # Terminal status
        if self.is_terminal():
            output.append("\n*** GAME OVER ***")
            winner = (
                1
                if self.scores[1] > self.scores[2]
                else (2 if self.scores[2] > self.scores[1] else None)
            )
            if winner:
                output.append(
                    f"Winner: Player {winner} with {self.scores[winner]} points!"
                )
            else:
                output.append("Game ended in a tie!")

        output.append("=" * 60)

        return "\n".join(output)

    @staticmethod
    def create_new_game() -> "ScrabbleState":
        """Factory function to create a new Scrabble game with initial setup."""
        # Initialize empty board
        board = [["" for _ in range(BOARD_SIZE)] for _ in range(BOARD_SIZE)]

        # Create tile pool with standard Scrabble distribution
        tile_distribution = {
            "A": 9,
            "B": 2,
            "C": 2,
            "D": 4,
            "E": 12,
            "F": 2,
            "G": 3,
            "H": 2,
            "I": 9,
            "J": 1,
            "K": 1,
            "L": 4,
            "M": 2,
            "N": 6,
            "O": 8,
            "P": 2,
            "Q": 1,
            "R": 6,
            "S": 4,
            "T": 6,
            "U": 4,
            "V": 2,
            "W": 2,
            "X": 1,
            "Y": 2,
            "Z": 1,
            "_": 2,  # '_' represents blank tiles
        }
        tile_pool = []
        for letter, count in tile_distribution.items():
            tile_pool.extend([letter] * count)
        random.shuffle(tile_pool)

        # Draw initial racks for both players
        racks = {}
        for player_id in [1, 2]:
            racks[player_id] = []
            for _ in range(RACK_SIZE):
                if tile_pool:
                    racks[player_id].append(tile_pool.pop())

        # Initialize scores
        scores = {1: 0, 2: 0}

        # Player 1 starts
        current_player = 1

        return ScrabbleState(
            board=board,
            tile_pool=tile_pool,
            racks=racks,
            current_player=current_player,
            scores=scores,
        )

    def get_legal_moves(self, player_id: int) -> List[Move]:
        if self.is_terminal():
            return []

        moves = []
        rack = self.racks[player_id]

        # Check if board is empty (first move)
        board_empty = all(
            self.board[r][c] == "" for r in range(BOARD_SIZE) for c in range(BOARD_SIZE)
        )

        # If board is empty, only allow moves through center (7, 7)
        if board_empty:
            moves.extend(self._generate_first_move(rack))
        else:
            # Generate moves that connect to existing tiles
            moves.extend(self._generate_connected_moves(rack))

        # Always allow passing as a move
        moves.append(Move([], 0, is_pass=True))

        return moves

    # --- BEGIN Game Move Logic ---

    def _generate_first_move(self, rack: List[str]) -> List[Move]:
        """Generate all valid first moves (must go through center square)."""
        moves = []
        center = 7

        # Try all permutations of rack tiles for horizontal and vertical placements

        for length in range(2, len(rack) + 1):
            for perm in permutations(rack, length):
                word = "".join(perm)
                if not is_valid_word(word):
                    continue

                # Try horizontal placements through center
                for start_col in range(
                    max(0, center - length + 1),
                    min(center + 1, BOARD_SIZE - length + 1),
                ):
                    if start_col <= center < start_col + length:
                        tiles_placed = [
                            (center, start_col + i, perm[i]) for i in range(length)
                        ]
                        score = self._calculate_score(tiles_placed)
                        moves.append(Move(tiles_placed, score))

                # Try vertical placements through center
                for start_row in range(
                    max(0, center - length + 1),
                    min(center + 1, BOARD_SIZE - length + 1),
                ):
                    if start_row <= center < start_row + length:
                        tiles_placed = [
                            (start_row + i, center, perm[i]) for i in range(length)
                        ]
                        score = self._calculate_score(tiles_placed)
                        moves.append(Move(tiles_placed, score))

        return moves

    def _generate_connected_moves(self, rack: List[str]) -> List[Move]:
        """Generate all moves that connect to existing tiles on the board."""
        moves = []

        # Find all anchor points (empty squares adjacent to filled squares)
        anchors = self._find_anchors()

        for anchor_row, anchor_col in anchors:
            # Try building words horizontally
            moves.extend(
                self._build_words_at_anchor(
                    rack, anchor_row, anchor_col, horizontal=True
                )
            )
            # Try building words vertically
            moves.extend(
                self._build_words_at_anchor(
                    rack, anchor_row, anchor_col, horizontal=False
                )
            )

        return moves

    def _find_anchors(self) -> List[Tuple[int, int]]:
        """Find all empty squares adjacent to filled squares."""
        anchors = []
        for r in range(BOARD_SIZE):
            for c in range(BOARD_SIZE):
                if self.board[r][c] == "":
                    # Check if adjacent to any filled square
                    adjacent_filled = False
                    for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                        nr, nc = r + dr, c + dc
                        if 0 <= nr < BOARD_SIZE and 0 <= nc < BOARD_SIZE:
                            if self.board[nr][nc] != "":
                                adjacent_filled = True
                                break
                    if adjacent_filled:
                        anchors.append((r, c))
        return anchors

    def _build_words_at_anchor(
        self, rack: List[str], anchor_row: int, anchor_col: int, horizontal: bool
    ) -> List[Move]:
        """Build all valid words that include the anchor square."""
        moves = []

        # For each possible word length
        for word_len in range(2, len(rack) + 1):
            # For each permutation of tiles
            for perm in permutations(rack, word_len):
                word = "".join(perm)
                if not is_valid_word(word):
                    continue

                # Try different positions where the anchor is part of the word
                for anchor_pos_in_word in range(word_len):
                    if horizontal:
                        start_col = anchor_col - anchor_pos_in_word
                        if start_col < 0 or start_col + word_len > BOARD_SIZE:
                            continue

                        # Build the placement
                        tiles_placed = []
                        valid = True
                        tiles_used_from_rack = []

                        for i in range(word_len):
                            row, col = anchor_row, start_col + i
                            if self.board[row][col] == "":
                                # Need to place a tile here
                                tiles_placed.append((row, col, perm[i]))
                                tiles_used_from_rack.append(perm[i])
                            elif self.board[row][col] == perm[i]:
                                # Tile already on board matches
                                continue
                            else:
                                # Conflict with existing tile
                                valid = False
                                break

                        if not valid or len(tiles_placed) == 0:
                            continue

                        # Check if we have the tiles in our rack
                        rack_copy = rack[:]
                        can_place = True
                        for tile in tiles_used_from_rack:
                            if tile in rack_copy:
                                rack_copy.remove(tile)
                            else:
                                can_place = False
                                break

                        if not can_place:
                            continue

                        # Validate the complete main word formed (including existing tiles)
                        temp_board = [row[:] for row in self.board]
                        for r, c, letter in tiles_placed:
                            temp_board[r][c] = letter
                        main_word = self._get_word_at(
                            temp_board, anchor_row, anchor_col, horizontal=True
                        )
                        if not is_valid_word(main_word):
                            continue

                        # Validate all cross words formed
                        if self._validate_placement(tiles_placed, horizontal):
                            score = self._calculate_score(tiles_placed)
                            moves.append(Move(tiles_placed, score))

                    else:  # vertical
                        start_row = anchor_row - anchor_pos_in_word
                        if start_row < 0 or start_row + word_len > BOARD_SIZE:
                            continue

                        # Build the placement
                        tiles_placed = []
                        valid = True
                        tiles_used_from_rack = []

                        for i in range(word_len):
                            row, col = start_row + i, anchor_col
                            if self.board[row][col] == "":
                                # Need to place a tile here
                                tiles_placed.append((row, col, perm[i]))
                                tiles_used_from_rack.append(perm[i])
                            elif self.board[row][col] == perm[i]:
                                # Tile already on board matches
                                continue
                            else:
                                # Conflict with existing tile
                                valid = False
                                break

                        if not valid or len(tiles_placed) == 0:
                            continue

                        # Check if we have the tiles in our rack
                        rack_copy = rack[:]
                        can_place = True
                        for tile in tiles_used_from_rack:
                            if tile in rack_copy:
                                rack_copy.remove(tile)
                            else:
                                can_place = False
                                break

                        if not can_place:
                            continue

                        # Validate the complete main word formed (including existing tiles)
                        temp_board = [row[:] for row in self.board]
                        for r, c, letter in tiles_placed:
                            temp_board[r][c] = letter
                        main_word = self._get_word_at(
                            temp_board, anchor_row, anchor_col, horizontal=False
                        )
                        if not is_valid_word(main_word):
                            continue

                        # Validate all cross words formed
                        if self._validate_placement(tiles_placed, horizontal):
                            score = self._calculate_score(tiles_placed)
                            moves.append(Move(tiles_placed, score))

        return moves

    def _validate_placement(
        self, tiles_placed: List[Tuple[int, int, str]], horizontal: bool
    ) -> bool:
        """Validate that all cross-words formed by placement are valid."""
        # Create temporary board with new tiles
        temp_board = [row[:] for row in self.board]
        for row, col, letter in tiles_placed:
            temp_board[row][col] = letter

        # Check each placed tile for cross-words
        for row, col, letter in tiles_placed:
            if horizontal:
                # Check vertical cross-word at this position
                cross_word = self._get_word_at(temp_board, row, col, horizontal=False)
                if len(cross_word) > 1 and not is_valid_word(cross_word):
                    return False
            else:
                # Check horizontal cross-word at this position
                cross_word = self._get_word_at(temp_board, row, col, horizontal=True)
                if len(cross_word) > 1 and not is_valid_word(cross_word):
                    return False

        return True

    def _get_word_at(
        self, board: List[List[str]], row: int, col: int, horizontal: bool
    ) -> str:
        """Extract the complete word at the given position."""
        if horizontal:
            # Find start of word
            start_col = col
            while start_col > 0 and board[row][start_col - 1] != "":
                start_col -= 1
            # Find end of word
            end_col = col
            while end_col < BOARD_SIZE - 1 and board[row][end_col + 1] != "":
                end_col += 1
            # Extract word
            return "".join(board[row][c] for c in range(start_col, end_col + 1))
        else:
            # Find start of word
            start_row = row
            while start_row > 0 and board[start_row - 1][col] != "":
                start_row -= 1
            # Find end of word
            end_row = row
            while end_row < BOARD_SIZE - 1 and board[end_row + 1][col] != "":
                end_row += 1
            # Extract word
            return "".join(board[r][col] for r in range(start_row, end_row + 1))

    def _calculate_score(self, tiles_placed: List[Tuple[int, int, str]]) -> int:
        """Calculate the score for placing these tiles."""
        if not tiles_placed:
            return 0

        # Determine direction of main word
        if len(tiles_placed) == 1:
            row, col, letter = tiles_placed[0]
            # Check if forms horizontal or vertical word with existing tiles
            horizontal_word = self._get_word_at(self.board, row, col, horizontal=True)
            vertical_word = self._get_word_at(self.board, row, col, horizontal=False)
            # For single tile, we need to consider both directions
            pass

        # Assume horizontal if all same row, vertical if all same column
        rows = [r for r, c, l in tiles_placed]
        cols = [c for r, c, l in tiles_placed]
        horizontal = len(set(rows)) == 1

        total_score = 0
        word_multiplier = 1

        # Create temp board for scoring
        temp_board = [row[:] for row in self.board]
        for row, col, letter in tiles_placed:
            temp_board[row][col] = letter

        # Score main word
        if horizontal:
            row = rows[0]
            start_col = min(cols)
            end_col = max(cols)
            # Extend to include existing tiles
            while start_col > 0 and temp_board[row][start_col - 1] != "":
                start_col -= 1
            while end_col < BOARD_SIZE - 1 and temp_board[row][end_col + 1] != "":
                end_col += 1

            word_score = 0
            for c in range(start_col, end_col + 1):
                letter = temp_board[row][c]
                letter_score = LETTER_VALUES.get(letter, 0)
                # Apply premium if tile was just placed
                if any(r == row and col == c for r, col, l in tiles_placed):
                    premium = PREMIUM_BOARD[row][c]
                    if premium == 2:  # Double letter
                        letter_score *= 2
                    elif premium == 3:  # Triple letter
                        letter_score *= 3
                    elif premium == 4:  # Double word
                        word_multiplier *= 2
                    elif premium == 5:  # Triple word
                        word_multiplier *= 3
                word_score += letter_score
            total_score += word_score * word_multiplier
        else:
            col = cols[0]
            start_row = min(rows)
            end_row = max(rows)
            # Extend to include existing tiles
            while start_row > 0 and temp_board[start_row - 1][col] != "":
                start_row -= 1
            while end_row < BOARD_SIZE - 1 and temp_board[end_row + 1][col] != "":
                end_row += 1

            word_score = 0
            for r in range(start_row, end_row + 1):
                letter = temp_board[r][col]
                letter_score = LETTER_VALUES.get(letter, 0)
                # Apply premium if tile was just placed
                if any(row == r and c == col for row, c, l in tiles_placed):
                    premium = PREMIUM_BOARD[r][col]
                    if premium == 2:  # Double letter
                        letter_score *= 2
                    elif premium == 3:  # Triple letter
                        letter_score *= 3
                    elif premium == 4:  # Double word
                        word_multiplier *= 2
                    elif premium == 5:  # Triple word
                        word_multiplier *= 3
                word_score += letter_score
            total_score += word_score * word_multiplier

        # Score cross-words
        for row, col, letter in tiles_placed:
            if horizontal:
                cross_word = self._get_word_at(temp_board, row, col, horizontal=False)
                if len(cross_word) > 1:
                    # Score this cross-word
                    start_row = row
                    while start_row > 0 and temp_board[start_row - 1][col] != "":
                        start_row -= 1
                    cross_score = 0
                    cross_multiplier = 1
                    for r in range(start_row, start_row + len(cross_word)):
                        l = temp_board[r][col]
                        ls = LETTER_VALUES.get(l, 0)
                        if r == row:  # This is the newly placed tile
                            premium = PREMIUM_BOARD[r][col]
                            if premium == 2:
                                ls *= 2
                            elif premium == 3:
                                ls *= 3
                            elif premium == 4:
                                cross_multiplier *= 2
                            elif premium == 5:
                                cross_multiplier *= 3
                        cross_score += ls
                    total_score += cross_score * cross_multiplier
            else:
                cross_word = self._get_word_at(temp_board, row, col, horizontal=True)
                if len(cross_word) > 1:
                    # Score this cross-word
                    start_col = col
                    while start_col > 0 and temp_board[row][start_col - 1] != "":
                        start_col -= 1
                    cross_score = 0
                    cross_multiplier = 1
                    for c in range(start_col, start_col + len(cross_word)):
                        l = temp_board[row][c]
                        ls = LETTER_VALUES.get(l, 0)
                        if c == col:  # This is the newly placed tile
                            premium = PREMIUM_BOARD[row][c]
                            if premium == 2:
                                ls *= 2
                            elif premium == 3:
                                ls *= 3
                            elif premium == 4:
                                cross_multiplier *= 2
                            elif premium == 5:
                                cross_multiplier *= 3
                        cross_score += ls
                    total_score += cross_score * cross_multiplier

        # Bonus for using all 7 tiles
        if len(tiles_placed) == 7:
            total_score += 50

        return total_score

    # --- END Game Move Logic ---

    def apply_move(self, move: Move) -> "ScrabbleState":
        """Creates and returns the next state after the move is applied."""
        opponent_id = 3 - self.current_player
        next_state = ScrabbleState(
            board=[row[:] for row in self.board],
            tile_pool=self.tile_pool[:],
            racks={p: r[:] for p, r in self.racks.items()},
            current_player=opponent_id,  # Switch player
            scores=self.scores.copy(),
        )

        player_performing_move = self.current_player

        if not move.is_pass:
            # Place tiles on the board
            for row, col, letter in move.tiles_placed:
                next_state.board[row][col] = letter

            # Update score
            next_state.scores[player_performing_move] += move.score

            # Remove used tiles from rack
            tiles_to_remove = [letter for _, _, letter in move.tiles_placed]
            rack_copy = next_state.racks[player_performing_move][:]
            for tile in tiles_to_remove:
                if tile in rack_copy:
                    rack_copy.remove(tile)

            # Draw new tiles
            tiles_used = len(move.tiles_placed)
            new_tiles = next_state._draw_tiles(tiles_used)

            # Update rack with remaining tiles plus new tiles
            next_state.racks[player_performing_move] = rack_copy + new_tiles

            next_state.passes_in_a_row = 0
        else:
            next_state.passes_in_a_row = self.passes_in_a_row + 1

        return next_state

    def _draw_tiles(self, count: int) -> List[str]:
        """Draws tiles from the pool (handles stochasticity)."""
        drawn = []
        # Draw up to 'count' tiles, or until pool is empty
        actual_draw_count = min(count, len(self.tile_pool))
        for _ in range(actual_draw_count):
            tile = random.choice(self.tile_pool)
            self.tile_pool.remove(tile)
            drawn.append(tile)
        return drawn

    def is_terminal(self) -> bool:
        """Game ends if tile pool is empty or two consecutive passes occur."""
        return len(self.tile_pool) == 0 or self.passes_in_a_row >= 2

    def get_utility(self, maximizing_player_id: int) -> float:
        """Returns the utility (score differential) from the perspective of the maximizing player.
        This makes the game zero-sum.
        """
        opponent_id = 3 - maximizing_player_id
        return self.scores[maximizing_player_id] - self.scores[opponent_id]


class AlphaBetaMinimax:
    """
    Implements a depth-limited Minimax search with Alpha-Beta Pruning.
    Crucial for demonstrating computational limits (high branching factor)
    and conceptual failures.
    """

    def __init__(self, max_depth: int):
        self.max_depth = max_depth
        self.nodes_explored = 0

    def find_best_move(self, state: ScrabbleState) -> Move:
        current_player = state.current_player
        self.nodes_explored = 0

        alpha = float("-inf")
        beta = float("inf")
        best_value = float("-inf")
        best_move = None

        moves = state.get_legal_moves(current_player)

        if not moves:
            return Move([], 0, is_pass=True)

        start_time = time.time()

        for move in moves:
            next_state = state.apply_move(move)
            # The subsequent layer is the Min player (opponent)
            value = self._minimax_value(
                next_state,
                self.max_depth - 1,
                alpha,
                beta,
                maximizing_player_id=current_player,
            )

            if value > best_value:
                best_value = value
                best_move = move

            # Update alpha at the root (Max level)
            alpha = max(alpha, best_value)

        duration = time.time() - start_time
        print(
            f"Minimax search completed in: {duration:.4f}s. Nodes explored: {self.nodes_explored}. Best move value: {best_value:.2f}"
        )

        return best_move

    def _minimax_value(
        self,
        state: ScrabbleState,
        depth: int,
        alpha: float,
        beta: float,
        maximizing_player_id: int,
    ) -> float:
        """Recursive Minimax function with Alpha-Beta pruning."""
        self.nodes_explored += 1

        if depth == 0 or state.is_terminal():
            # Return static evaluation (utility) at cutoff or terminal node
            return state.get_utility(maximizing_player_id)

        current_player_to_move = state.current_player
        is_max_player_turn = current_player_to_move == maximizing_player_id

        moves = state.get_legal_moves(current_player_to_move)

        if is_max_player_turn:
            value = float("-inf")
            for move in moves:
                next_state = state.apply_move(move)
                value = max(
                    value,
                    self._minimax_value(
                        next_state, depth - 1, alpha, beta, maximizing_player_id
                    ),
                )
                alpha = max(alpha, value)
                if alpha >= beta:
                    break  # Beta cutoff (Pruning)
            return value

        else:  # Min player turn (opponent)
            value = float("inf")
            for move in moves:
                next_state = state.apply_move(move)
                value = min(
                    value,
                    self._minimax_value(
                        next_state, depth - 1, alpha, beta, maximizing_player_id
                    ),
                )
                beta = min(beta, value)
                if alpha >= beta:
                    break  # Alpha cutoff (Pruning)
            return value


class MonteCarlo:
    """
    Framework for Adversarial Monte Carlo Tree Search (MCTS) for Scrabble.
    This implementation focuses on the Monte Carlo aspect.
    """

    def __init__(self, num_playouts: int, heuristic_fn):
        self.num_playouts = num_playouts
        self.heuristic_fn = heuristic_fn

    async def find_best_move(self, state: ScrabbleState) -> Move:
        """Runs the Monte Carlo simulation to estimate move values."""

        moves = state.get_legal_moves(state.current_player)
        if not moves:
            return Move([], 0, is_pass=True)

        move_values = {}

        # Determine the number of playouts per move
        playouts_per_move = int(self.num_playouts // len(moves)) or 1

        print(
            f"Starting MCTS: {playouts_per_move} playouts per move (for {len(moves)} possible moves)..."
        )

        for move in moves:
            total_value = 0

            # Run simulations (rollouts)
            for _ in range(playouts_per_move):
                next_state = state.apply_move(move)

                # Use the agentic heuristic to evaluate the outcome of the simulation
                # NOTE: A real MCTS playout runs until terminal state, but here we
                # use the heuristic to evaluate the strategic value of the immediate successor
                # state resulting from the move, simulating guidance.
                value = await self.heuristic_fn(next_state, state.current_player)
                total_value += value

            avg_value = total_value / playouts_per_move
            move_values[move] = avg_value

        if move_values:
            best_move = max(move_values, key=move_values.get)
            return best_move

        return Move([], 0, is_pass=True)

Overwriting L5_utils.py


In [19]:
# @title
%%writefile words.txt
that
this
with
from
your
have
more
will
home
about
page
search
free
other
information
time
they
site
what
which
their
news
there
only
when
contact
here
business
also
help
view
online
first
been
would
were
services
some
these
click
like
service
than
find
price
date
back
people
list
name
just
over
state
year
into
email
health
world
next
used
work
last
most
products
music
data
make
them
should
product
system
post
city
policy
number
such
please
available
copyright
support
message
after
best
software
then
good
video
well
where
info
rights
public
books
high
school
through
each
links
review
years
order
very
privacy
book
items
company
read
group
need
many
user
said
does
under
general
research
university
january
mail
full
reviews
program
life
know
games
days
management
part
could
great
united
hotel
real
item
international
center
ebay
must
store
travel
comments
made
development
report
member
details
line
terms
before
hotels
send
right
type
because
local
those
using
results
office
education
national
design
take
posted
internet
address
community
within
states
area
want
phone
shipping
reserved
subject
between
forum
family
long
based
code
show
even
black
check
special
prices
website
index
being
women
much
sign
file
link
open
today
technology
south
case
project
same
pages
version
section
found
sports
house
related
security
both
county
american
photo
game
members
power
while
care
network
down
computer
systems
three
total
place
following
download
without
access
think
north
resources
current
posts
media
control
water
history
pictures
size
personal
since
including
guide
shop
directory
board
location
change
white
text
small
rating
rate
government
children
during
return
students
shopping
account
times
sites
level
digital
profile
previous
form
events
love
john
main
call
hours
image
department
title
description
insurance
another
shall
property
class
still
money
quality
every
listing
content
country
private
little
visit
save
tools
reply
customer
december
compare
movies
include
college
value
article
york
card
jobs
provide
food
source
author
different
press
learn
sale
around
print
course
canada
process
teen
room
stock
training
credit
point
join
science
categories
advanced
west
sales
look
english
left
team
estate
conditions
select
windows
photos
thread
week
category
note
live
large
gallery
table
register
however
june
october
november
market
library
really
action
start
series
model
features
industry
plan
human
provided
required
second
accessories
cost
movie
forums
march
september
better
questions
july
yahoo
going
medical
test
friend
come
server
study
application
cart
staff
articles
feedback
again
play
looking
issues
april
never
users
complete
street
topic
comment
financial
things
working
against
standard
person
below
mobile
less
blog
party
payment
equipment
login
student
programs
offers
legal
above
recent
park
stores
side
problem
give
memory
performance
social
august
quote
language
story
sell
options
experience
rates
create
body
young
america
important
field
east
paper
single
activities
club
example
girls
additional
password
latest
something
road
gift
question
changes
night
hard
texas
four
poker
status
browse
issue
range
building
seller
court
february
always
result
audio
light
write
offer
blue
groups
easy
given
files
event
release
analysis
request
china
making
picture
needs
possible
might
professional
month
major
star
areas
future
space
committee
hand
cards
problems
london
washington
meeting
become
interest
child
keep
enter
california
share
similar
garden
schools
million
added
reference
companies
listed
baby
learning
energy
delivery
popular
term
film
stories
computers
journal
reports
welcome
central
images
president
notice
original
head
radio
until
cell
color
self
council
away
includes
track
australia
discussion
archive
once
others
entertainment
agreement
format
least
society
months
safety
friends
sure
trade
edition
cars
messages
marketing
tell
further
updated
association
able
having
provides
david
already
green
studies
close
common
drive
specific
several
gold
living
collection
called
short
arts
display
limited
powered
solutions
means
director
daily
beach
past
natural
whether
electronics
five
upon
period
planning
database
says
official
weather
land
average
done
technical
window
france
region
island
record
direct
microsoft
conference
environment
records
district
calendar
costs
style
front
statement
update
parts
ever
downloads
early
miles
sound
resource
present
applications
either
document
word
works
material
bill
written
talk
federal
hosting
rules
final
adult
tickets
thing
centre
requirements
cheap
kids
finance
true
minutes
else
mark
third
rock
gifts
europe
reading
topics
individual
tips
plus
auto
cover
usually
edit
together
videos
percent
fast
function
fact
unit
getting
global
tech
meet
economic
player
projects
lyrics
often
subscribe
submit
germany
amount
watch
included
feel
though
bank
risk
thanks
everything
deals
various
words
linux
production
commercial
james
weight
town
heart
advertising
received
choose
treatment
newsletter
archives
points
knowledge
magazine
error
camera
girl
currently
construction
toys
registered
clear
golf
receive
domain
methods
chapter
makes
protection
policies
loan
wide
beauty
manager
india
position
taken
sort
listings
models
michael
known
half
cases
step
engineering
florida
simple
quick
none
wireless
license
paul
friday
lake
whole
annual
published
later
basic
sony
shows
corporate
google
church
method
purchase
customers
active
response
practice
hardware
figure
materials
fire
holiday
chat
enough
designed
along
among
death
writing
speed
html
countries
loss
face
brand
discount
higher
effects
created
remember
standards
yellow
political
increase
advertise
kingdom
base
near
environmental
thought
stuff
french
storage
japan
doing
loans
shoes
entry
stay
nature
orders
availability
africa
summary
turn
mean
growth
notes
agency
king
monday
european
activity
copy
although
drug
pics
western
income
force
cash
employment
overall
river
commission
package
contents
seen
players
engine
port
album
regional
stop
supplies
started
administration
institute
views
plans
double
build
screen
exchange
types
soon
sponsored
lines
electronic
continue
across
benefits
needed
season
apply
someone
held
anything
printer
condition
effective
believe
organization
effect
asked
mind
sunday
selection
casino
lost
tour
menu
volume
cross
anyone
mortgage
hope
silver
corporation
wish
inside
solution
mature
role
rather
weeks
addition
came
supply
nothing
certain
executive
running
lower
necessary
union
jewelry
according
clothing
particular
fine
names
robert
homepage
hour
skills
bush
islands
advice
career
military
rental
decision
leave
british
teens
huge
woman
facilities
kind
sellers
middle
move
cable
opportunities
taking
values
division
coming
tuesday
object
lesbian
appropriate
machine
logo
length
actually
nice
score
statistics
client
returns
capital
follow
sample
investment
sent
shown
saturday
christmas
england
culture
band
flash
lead
george
choice
went
starting
registration
thursday
courses
consumer
airport
foreign
artist
outside
furniture
levels
channel
letter
mode
phones
ideas
wednesday
structure
fund
summer
allow
degree
contract
button
releases
homes
super
male
matter
custom
virginia
almost
took
located
multiple
asian
distribution
editor
industrial
cause
potential
song
cnet
focus
late
fall
featured
idea
rooms
female
responsible
communications
associated
thomas
primary
cancer
numbers
reason
tool
browser
spring
foundation
answer
voice
friendly
schedule
documents
communication
purpose
feature
comes
police
everyone
independent
approach
cameras
brown
physical
operating
hill
maps
medicine
deal
hold
ratings
chicago
forms
glass
happy
smith
wanted
developed
thank
safe
unique
survey
prior
telephone
sport
ready
feed
animal
sources
mexico
population
regular
secure
navigation
operations
therefore
simply
evidence
station
christian
round
paypal
favorite
understand
option
master
valley
recently
probably
rentals
built
publications
blood
worldwide
improve
connection
publisher
hall
larger
anti
networks
earth
parents
nokia
impact
transfer
introduction
kitchen
strong
carolina
wedding
properties
hospital
ground
overview
ship
accommodation
owners
disease
excellent
paid
italy
perfect
hair
opportunity
classic
basis
command
cities
william
express
award
distance
tree
peter
assessment
ensure
thus
wall
involved
extra
especially
interface
partners
budget
rated
guides
success
maximum
operation
existing
quite
selected
amazon
patients
restaurants
beautiful
warning
wine
locations
horse
vote
forward
flowers
stars
significant
lists
technologies
owner
retail
animals
useful
directly
manufacturer
ways
providing
rule
housing
takes
bring
catalog
searches
trying
mother
authority
considered
told
traffic
programme
joined
input
strategy
feet
agent
valid
modern
senior
ireland
teaching
door
grand
testing
trial
charge
units
instead
canadian
cool
normal
wrote
enterprise
ships
entire
educational
leading
metal
positive
fitness
chinese
opinion
asia
football
abstract
uses
output
funds
greater
likely
develop
employees
artists
alternative
processing
responsibility
resolution
java
guest
seems
publication
pass
relations
trust
contains
session
multi
photography
republic
fees
components
vacation
century
academic
assistance
completed
skin
graphics
indian
prev
mary
expected
ring
grade
dating
pacific
mountain
organizations
filter
mailing
vehicle
longer
consider
northern
behind
panel
floor
german
buying
match
proposed
default
require
iraq
boys
outdoor
deep
morning
otherwise
allows
rest
protein
plant
reported
transportation
pool
mini
politics
partner
disclaimer
authors
boards
faculty
parties
fish
membership
mission
string
sense
modified
pack
released
stage
internal
goods
recommended
born
unless
richard
detailed
japanese
race
approved
background
target
except
character
maintenance
ability
maybe
functions
moving
brands
places
pretty
trademarks
phentermine
spain
southern
yourself
winter
battery
youth
pressure
submitted
boston
debt
keywords
medium
television
interested
core
break
purposes
throughout
sets
dance
wood
itself
defined
papers
playing
awards
studio
reader
virtual
device
established
answers
rent
remote
dark
programming
external
apple
regarding
instructions
offered
theory
enjoy
remove
surface
minimum
visual
host
variety
teachers
isbn
martin
manual
block
subjects
agents
increased
repair
fair
civil
steel
understanding
songs
fixed
wrong
beginning
hands
associates
finally
updates
desktop
classes
paris
ohio
gets
sector
capacity
requires
jersey
fully
father
electric
instruments
quotes
officer
driver
businesses
dead
respect
unknown
specified
restaurant
mike
trip
worth
procedures
poor
teacher
eyes
relationship
workers
farm
georgia
peace
traditional
campus
showing
creative
coast
benefit
progress
funding
devices
lord
grant
agree
fiction
hear
sometimes
watches
careers
beyond
goes
families
museum
themselves
transport
interesting
blogs
wife
evaluation
accepted
former
implementation
hits
zone
complex
galleries
references
presented
jack
flat
flow
agencies
literature
respective
parent
spanish
michigan
columbia
setting
scale
stand
economy
highest
helpful
monthly
critical
frame
musical
definition
secretary
angeles
networking
path
australian
employee
chief
gives
bottom
magazines
packages
detail
francisco
laws
changed
heard
begin
individuals
colorado
royal
clean
switch
russian
largest
african
titles
relevant
guidelines
justice
connect
bible
basket
applied
weekly
installation
described
demand
suite
vegas
square
chris
attention
advance
skip
diet
army
auction
gear
difference
allowed
correct
charles
nation
selling
lots
piece
sheet
firm
seven
older
illinois
regulations
elements
species
jump
cells
module
resort
facility
random
pricing
dvds
certificate
minister
motion
looks
fashion
directions
visitors
documentation
monitor
trading
forest
calls
whose
coverage
couple
giving
chance
vision
ball
ending
clients
actions
listen
discuss
accept
automotive
naked
goal
successful
sold
wind
communities
clinical
situation
sciences
markets
lowest
highly
publishing
appear
emergency
developing
lives
currency
leather
determine
temperature
palm
announcements
patient
actual
historical
stone
commerce
ringtones
perhaps
persons
difficult
scientific
satellite
tests
village
accounts
amateur
pain
xbox
particularly
factors
coffee
settings
buyer
cultural
steve
easily
oral
ford
poster
edge
functional
root
closed
holidays
pink
zealand
balance
monitoring
graduate
replies
shot
architecture
initial
label
thinking
scott
recommend
canon
league
waste
minute
provider
optional
dictionary
cold
accounting
manufacturing
sections
chair
fishing
effort
phase
fields
fantasy
letters
motor
professor
context
install
shirt
apparel
generally
continued
foot
mass
crime
count
breast
techniques
johnson
quickly
dollars
websites
religion
claim
driving
permission
surgery
patch
heat
wild
measures
generation
kansas
miss
chemical
doctor
task
reduce
brought
himself
component
enable
exercise
santa
guarantee
leader
diamond
israel
processes
soft
servers
alone
meetings
seconds
jones
arizona
keyword
interests
flight
congress
fuel
username
walk
produced
italian
paperback
classifieds
wait
supported
pocket
saint
rose
freedom
argument
competition
creating
drugs
joint
premium
providers
fresh
characters
attorney
upgrade
factor
growing
thousands
stream
apartments
pick
hearing
eastern
auctions
therapy
entries
dates
generated
signed
upper
administrative
serious
prime
samsung
limit
began
louis
steps
errors
shops
efforts
informed
thoughts
creek
worked
quantity
urban
practices
sorted
reporting
essential
myself
tours
platform
load
affiliate
labor
immediately
admin
nursing
defense
machines
designated
tags
heavy
covered
recovery
guys
integrated
configuration
merchant
comprehensive
expert
universal
protect
drop
solid
presentation
languages
became
orange
compliance
vehicles
prevent
theme
rich
campaign
marine
improvement
guitar
finding
pennsylvania
examples
ipod
saying
spirit
claims
challenge
motorola
acceptance
strategies
seem
affairs
touch
intended
towards
goals
hire
election
suggest
branch
charges
serve
affiliates
reasons
magic
mount
smart
talking
gave
ones
latin
multimedia
avoid
certified
manage
corner
rank
computing
oregon
element
birth
virus
abuse
interactive
requests
separate
quarter
procedure
leadership
tables
define
racing
religious
facts
breakfast
kong
column
plants
faith
chain
developer
identify
avenue
missing
died
approximately
domestic
sitemap
recommendations
moved
houston
reach
comparison
mental
viewed
moment
extended
sequence
inch
attack
sorry
centers
opening
damage
reserve
recipes
gamma
plastic
produce
snow
placed
truth
counter
failure
follows
weekend
dollar
camp
ontario
automatically
minnesota
films
bridge
native
fill
williams
movement
printing
baseball
owned
approval
draft
chart
played
contacts
jesus
readers
clubs
jackson
equal
adventure
matching
offering
shirts
profit
leaders
posters
institutions
assistant
variable
advertisement
expect
parking
headlines
yesterday
compared
determined
wholesale
workshop
russia
gone
codes
kinds
extension
seattle
statements
golden
completely
teams
fort
lighting
senate
forces
funny
brother
gene
turned
portable
tried
electrical
applicable
disc
returned
pattern
boat
named
theatre
laser
earlier
manufacturers
sponsor
classical
icon
warranty
dedicated
indiana
direction
harry
basketball
objects
ends
delete
evening
assembly
nuclear
taxes
mouse
signal
criminal
issued
brain
sexual
wisconsin
powerful
dream
obtained
false
cast
flower
felt
personnel
passed
supplied
identified
falls
soul
aids
opinions
promote
stated
stats
hawaii
professionals
appears
carry
flag
decided
covers
advantage
hello
designs
maintain
tourism
priority
newsletters
adults
clips
savings
graphic
atom
payments
estimated
binding
brief
ended
winning
eight
anonymous
iron
straight
script
served
wants
miscellaneous
prepared
void
dining
alert
integration
atlanta
dakota
interview
framework
disk
installed
queen
credits
clearly
handle
sweet
desk
criteria
pubmed
dave
massachusetts
diego
hong
vice
associate
truck
behavior
enlarge
frequently
revenue
measure
changing
votes
duty
looked
discussions
bear
gain
festival
laboratory
ocean
flights
experts
signs
lack
depth
iowa
whatever
logged
laptop
vintage
train
exactly
explore
maryland
concept
nearly
eligible
checkout
reality
forgot
handling
origin
knew
gaming
feeds
billion
destination
scotland
faster
intelligence
dallas
bought
nations
route
followed
specifications
broken
tripadvisor
frank
alaska
zoom
blow
battle
residential
anime
speak
decisions
industries
protocol
query
clip
partnership
editorial
expression
equity
provisions
speech
wire
principles
suggestions
rural
shared
sounds
replacement
tape
strategic
judge
spam
economics
acid
bytes
cent
forced
compatible
fight
apartment
height
null
zero
speaker
filed
netherlands
obtain
consulting
recreation
offices
designer
remain
managed
failed
marriage
roll
korea
banks
participants
secret
bath
kelly
leads
negative
austin
favorites
toronto
theater
springs
missouri
andrew
perform
healthy
translation
estimates
font
assets
injury
joseph
ministry
drivers
lawyer
figures
married
protected
proposal
sharing
philadelphia
portal
waiting
birthday
beta
fail
gratis
banking
officials
brian
toward
slightly
assist
conduct
contained
lingerie
legislation
calling
parameters
jazz
serving
bags
profiles
miami
comics
matters
houses
postal
relationships
tennessee
wear
controls
breaking
combined
ultimate
wales
representative
frequency
introduced
minor
finish
departments
residents
noted
displayed
reduced
physics
rare
spent
performed
extreme
samples
davis
daniel
bars
reviewed
forecast
removed
helps
singles
administrator
cycle
amounts
contain
accuracy
dual
rise
sleep
bird
pharmacy
brazil
creation
static
scene
hunter
addresses
lady
crystal
famous
writer
chairman
violence
fans
oklahoma
speakers
drink
academy
dynamic
gender
permanent
agriculture
dell
cleaning
constitutes
portfolio
practical
delivered
collectibles
infrastructure
exclusive
seat
concerns
colour
vendor
originally
intel
utilities
philosophy
regulation
officers
reduction
bids
referred
supports
nutrition
recording
regions
junior
toll
cape
rings
meaning
secondary
wonderful
mine
ladies
henry
ticket
announced
guess
agreed
prevention
whom
soccer
math
import
posting
presence
instant
mentioned
automatic
healthcare
viewing
maintained
increasing
majority
connected
christ
dogs
directors
aspects
austria
ahead
moon
participation
scheme
utility
preview
manner
matrix
containing
combination
devel
amendment
despite
strength
guaranteed
turkey
libraries
proper
distributed
degrees
singapore
enterprises
delta
fear
seeking
inches
phoenix
convention
shares
principal
daughter
standing
comfort
colors
wars
cisco
ordering
kept
alpha
appeal
cruise
bonus
certification
previously
bookmark
buildings
specials
beat
disney
household
batteries
adobe
smoking
becomes
drives
arms
alabama
improved
trees
achieve
positions
dress
subscription
dealer
contemporary
utah
nearby
carried
happen
exposure
panasonic
hide
permalink
signature
gambling
refer
miller
provision
outdoors
clothes
caused
luxury
babes
frames
certainly
indeed
newspaper
circuit
layer
printed
slow
removal
easier
liability
trademark
printers
faqs
nine
adding
kentucky
mostly
eric
spot
taylor
trackback
prints
spend
factory
interior
revised
grow
americans
optical
promotion
relative
amazing
clock
identity
suites
conversion
feeling
hidden
reasonable
victoria
serial
relief
revision
broadband
influence
ratio
importance
rain
onto
planet
webmaster
copies
recipe
permit
seeing
proof
diff
tennis
bass
prescription
bedroom
empty
instance
hole
pets
ride
licensed
orlando
specifically
bureau
maine
represent
conservation
pair
ideal
specs
recorded
pieces
finished
parks
dinner
lawyers
sydney
stress
cream
runs
trends
yeah
discover
patterns
boxes
louisiana
hills
javascript
fourth
advisor
marketplace
evil
aware
wilson
shape
evolution
irish
certificates
objectives
stations
suggested
remains
greatest
firms
concerned
euro
operator
structures
generic
encyclopedia
usage
charts
continuing
mixed
census
interracial
peak
competitive
exist
wheel
transit
suppliers
salt
compact
poetry
lights
tracking
angel
bell
keeping
preparation
attempt
receiving
matches
accordance
width
noise
engines
forget
array
discussed
accurate
stephen
elizabeth
climate
reservations
playstation
alcohol
greek
instruction
managing
annotation
sister
differences
walking
explain
smaller
newest
establish
happened
expressed
jeff
extent
sharp
lesbians
lane
paragraph
kill
mathematics
compensation
export
managers
aircraft
modules
sweden
conflict
conducted
versions
employer
occur
percentage
knows
mississippi
describe
concern
backup
requested
citizens
connecticut
heritage
personals
immediate
holding
trouble
spread
coach
kevin
agricultural
expand
supporting
audience
assigned
jordan
collections
ages
participate
plug
specialist
cook
affect
virgin
experienced
investigation
raised
institution
directed
dealers
searching
sporting
helping
perl
affected
bike
totally
plate
expenses
indicate
blonde
proceedings
favourite
transmission
anderson
characteristics
lose
organic
seek
experiences
albums
cheats
extremely
verzeichnis
contracts
guests
hosted
diseases
concerning
developers
equivalent
chemistry
tony
neighborhood
nevada
kits
thailand
variables
agenda
anyway
continues
tracks
advisory
curriculum
logic
template
prince
circle
soil
grants
anywhere
psychology
responses
atlantic
circumstances
edward
investor
identification
leaving
wildlife
appliances
matt
elementary
cooking
speaking
sponsors
unlimited
respond
sizes
plain
exit
entered
iran
keys
launch
wave
checking
costa
belgium
printable
holy
acts
guidance
mesh
trail
enforcement
symbol
crafts
highway
buddy
hardcover
observed
dean
setup
poll
booking
glossary
fiscal
celebrity
styles
denver
unix
filled
bond
channels
ericsson
appendix
notify
blues
chocolate
portion
scope
hampshire
supplier
cables
cotton
bluetooth
controlled
requirement
authorities
biology
dental
killed
border
ancient
debate
representatives
starts
pregnancy
causes
arkansas
biography
leisure
attractions
learned
transactions
notebook
explorer
historic
attached
opened
husband
disabled
authorized
crazy
upcoming
britain
concert
retirement
scores
financing
efficiency
comedy
adopted
efficient
weblog
linear
commitment
specialty
bears
jean
carrier
edited
constant
visa
mouth
jewish
meter
linked
portland
interviews
concepts
reflect
pure
deliver
wonder
lessons
fruit
begins
qualified
reform
lens
alerts
treated
discovery
draw
mysql
classified
relating
assume
confidence
alliance
confirm
warm
neither
lewis
howard
offline
leaves
engineer
lifestyle
consistent
replace
clearance
connections
inventory
converter
organisation
babe
checks
reached
becoming
safari
objective
indicated
sugar
crew
legs
stick
securities
allen
relation
enabled
genre
slide
montana
volunteer
tested
rear
democratic
enhance
switzerland
exact
bound
parameter
adapter
processor
node
formal
dimensions
contribute
lock
hockey
storm
micro
colleges
laptops
mile
showed
challenges
editors
mens
threads
bowl
supreme
brothers
recognition
presents
tank
submission
dolls
estimate
encourage
navy
regulatory
inspection
consumers
cancel
limits
territory
transaction
manchester
weapons
paint
delay
pilot
outlet
contributions
continuous
czech
resulting
cambridge
initiative
novel
execution
disability
increases
ultra
winner
idaho
contractor
episode
examination
potter
dish
plays
bulletin
indicates
modify
oxford
adam
truly
epinions
painting
committed
extensive
affordable
universe
candidate
databases
patent
slot
outstanding
eating
perspective
planned
watching
lodge
messenger
mirror
tournament
consideration
discounts
sterling
sessions
kernel
stocks
buyers
journals
gray
catalogue
jennifer
antonio
charged
broad
taiwan
chosen
demo
greece
swiss
sarah
clark
labour
hate
terminal
publishers
nights
behalf
caribbean
liquid
rice
nebraska
loop
salary
reservation
foods
gourmet
guard
properly
orleans
saving
remaining
empire
resume
twenty
newly
raise
prepare
avatar
gary
depending
illegal
expansion
vary
hundreds
rome
arab
lincoln
helped
premier
tomorrow
purchased
milk
decide
consent
drama
visiting
performing
downtown
keyboard
contest
collected
bands
boot
suitable
absolutely
millions
lunch
audit
push
chamber
guinea
findings
muscle
featuring
implement
clicking
scheduled
polls
typical
tower
yours
misc
calculator
significantly
chicken
temporary
attend
shower
alan
sending
jason
tonight
dear
sufficient
holdem
shell
province
catholic
awareness
vancouver
governor
beer
seemed
contribution
measurement
swimming
spyware
formula
constitution
packaging
solar
jose
catch
jane
pakistan
reliable
consultation
northwest
doubt
earn
finder
unable
periods
classroom
tasks
democracy
attacks
wallpaper
merchandise
const
resistance
doors
symptoms
resorts
biggest
memorial
visitor
twin
forth
insert
baltimore
gateway
dont
alumni
drawing
candidates
charlotte
ordered
biological
fighting
transition
happens
preferences
romance
instrument
bruce
split
themes
powers
heaven
bits
pregnant
twice
classification
focused
egypt
physician
hollywood
bargain
wikipedia
cellular
norway
vermont
asking
blocks
normally
spiritual
hunting
diabetes
suit
shift
chip
bodies
photographs
cutting
simon
writers
marks
flexible
loved
favourites
mapping
numerous
relatively
birds
satisfaction
represents
char
indexed
pittsburgh
superior
preferred
saved
paying
cartoon
shots
intellectual
moore
granted
choices
carbon
spending
comfortable
magnetic
interaction
listening
effectively
registry
crisis
outlook
massive
denmark
employed
bright
treat
header
poverty
formed
piano
echo
grid
sheets
patrick
experimental
puerto
revolution
consolidation
displays
plasma
allowing
earnings
voip
mystery
landscape
dependent
mechanical
journey
delaware
bidding
consultants
risks
banner
applicant
charter
barbara
cooperation
counties
acquisition
ports
implemented
directories
recognized
dreams
blogger
notification
licensing
stands
teach
occurred
textbooks
rapid
pull
hairy
diversity
cleveland
reverse
deposit
seminar
investments
latina
nasa
wheels
specify
accessibility
dutch
sensitive
templates
formats
depends
boots
holds
router
concrete
editing
poland
folder
womens
completion
upload
pulse
universities
technique
contractors
voting
courts
notices
subscriptions
calculate
detroit
alexander
broadcast
converted
metro
toshiba
anniversary
improvements
strip
specification
pearl
accident
nick
accessible
accessory
resident
plot
possibly
airline
typically
representation
regard
pump
exists
arrangements
smooth
conferences
uniprotkb
strike
consumption
birmingham
flashing
narrow
afternoon
threat
surveys
sitting
putting
consultant
controller
ownership
committees
legislative
researchers
vietnam
trailer
anne
castle
gardens
missed
malaysia
unsubscribe
antique
labels
willing
molecular
acting
heads
stored
exam
logos
residence
attorneys
antiques
density
hundred
ryan
operators
strange
sustainable
philippines
statistical
beds
mention
innovation
employers
grey
parallel
honda
amended
operate
bills
bold
bathroom
stable
opera
definitions
doctors
lesson
cinema
asset
scan
elections
drinking
reaction
blank
enhanced
entitled
severe
generate
stainless
newspapers
hospitals
deluxe
humor
aged
monitors
exception
lived
duration
bulk
successfully
indonesia
pursuant
fabric
visits
primarily
tight
domains
capabilities
pmid
contrast
recommendation
flying
recruitment
berlin
cute
organized
para
siemens
adoption
improving
expensive
meant
capture
pounds
buffalo
organisations
plane
explained
seed
programmes
desire
expertise
mechanism
camping
jewellery
meets
welfare
peer
caught
eventually
marked
driven
measured
medline
bottle
agreements
considering
innovative
marshall
massage
rubber
conclusion
closing
tampa
thousand
meat
legend
grace
susan
adams
python
monster
alex
bang
villa
bone
columns
disorders
bugs
collaboration
hamilton
detection
cookies
inner
formation
tutorial
engineers
entity
cruises
gate
holder
proposals
moderator
tutorials
settlement
portugal
lawrence
roman
duties
valuable
tone
collectables
ethics
forever
dragon
busy
captain
fantastic
imagine
brings
heating
neck
wing
governments
purchasing
scripts
stereo
appointed
taste
dealing
commit
tiny
operational
rail
airlines
liberal
livecam
trips
sides
tube
turns
corresponding
descriptions
cache
belt
jacket
determination
animation
oracle
matthew
lease
productions
aviation
hobbies
proud
excess
disaster
console
commands
telecommunications
instructor
giant
achieved
injuries
shipped
seats
approaches
alarm
voltage
anthony
nintendo
usual
loading
stamps
appeared
franklin
angle
vinyl
highlights
mining
designers
melbourne
ongoing
worst
imaging
betting
scientists
liberty
wyoming
blackjack
argentina
convert
possibility
analyst
commissioner
dangerous
garage
exciting
reliability
thongs
unfortunately
respectively
volunteers
attachment
ringtone
finland
morgan
derived
pleasure
honor
oriented
eagle
desktops
pants
columbus
nurse
prayer
appointment
workshops
hurricane
quiet
luck
postage
producer
represented
mortgages
dial
responsibilities
cheese
comic
carefully
productivity
investors
crown
underground
diagnosis
maker
crack
principle
picks
vacations
gang
semester
calculated
fetish
applies
casinos
appearance
smoke
apache
filters
incorporated
craft
cake
notebooks
apart
fellow
blind
lounge
algorithm
semi
coins
andy
gross
strongly
cafe
valentine
hilton
proteins
horror
familiar
capable
douglas
debian
till
involving
investing
christopher
admission
epson
shoe
elected
carrying
victory
sand
madison
terrorism
editions
mainly
ethnic
parliament
actor
finds
seal
situations
fifth
allocated
citizen
vertical
corrections
structural
municipal
describes
prize
occurs
absolute
disabilities
consists
anytime
substance
prohibited
addressed
lies
pipe
soldiers
guardian
lecture
simulation
layout
initiatives
concentration
classics
interpretation
horses
dirty
deck
wayne
donate
taught
bankruptcy
worker
optimization
alive
temple
substances
prove
discovered
wings
breaks
genetic
restrictions
participating
waters
promise
thin
exhibition
prefer
ridge
cabinet
modem
harris
bringing
sick
dose
evaluate
tiffany
tropical
collect
composition
toyota
streets
nationwide
vector
definitely
shaved
turning
buffer
purple
existence
commentary
larry
limousines
developments
immigration
destinations
lets
mutual
pipeline
necessarily
syntax
attribute
prison
skill
chairs
everyday
apparently
surrounding
mountains
moves
popularity
inquiry
ethernet
checked
exhibit
throw
trend
sierra
visible
cats
desert
postposted
oldest
rhode
coordinator
obviously
mercury
steven
handbook
greg
navigate
worse
summit
victims
spaces
fundamental
burning
escape
coupons
somewhat
receiver
substantial
progressive
cialis
boats
glance
scottish
championship
arcade
richmond
sacramento
impossible
russell
tells
obvious
fiber
depression
graph
covering
platinum
judgment
bedrooms
talks
filing
foster
modeling
passing
awarded
testimonials
trials
tissue
memorabilia
clinton
masters
bonds
cartridge
alberta
explanation
folk
commons
cincinnati
subsection
fraud
electricity
permitted
spectrum
arrival
okay
pottery
emphasis
roger
aspect
workplace
awesome
mexican
confirmed
counts
priced
wallpapers
hist
crash
lift
desired
inter
closer
assumes
heights
shadow
riding
infection
firefox
lisa
expense
grove
eligibility
venture
clinic
korean
healing
princess
mall
entering
packet
spray
studios
involvement
buttons
placement
observations
vbulletin
funded
thompson
winners
extend
roads
subsequent
dublin
rolling
fell
motorcycle
yard
disclosure
establishment
memories
nelson
arrived
creates
faces
tourist
mayor
murder
sean
adequate
senator
yield
presentations
grades
cartoons
pour
digest
lodging
tion
dust
hence
wiki
entirely
replaced
radar
rescue
undergraduate
losses
combat
reducing
stopped
occupation
lakes
donations
associations
citysearch
closely
radiation
diary
seriously
kings
shooting
kent
adds
flags
baker
launched
elsewhere
pollution
conservative
guestbook
shock
effectiveness
walls
abroad
ebony
ward
drawn
arthur
visited
roof
walker
demonstrate
atmosphere
suggests
kiss
beast
operated
experiment
targets
overseas
purchases
dodge
counsel
federation
pizza
invited
yards
assignment
chemicals
gordon
farmers
queries
rush
ukraine
absence
nearest
cluster
vendors
mpeg
whereas
yoga
serves
woods
surprise
lamp
rico
partial
shoppers
phil
everybody
couples
nashville
ranking
jokes
http
simpson
twiki
sublime
counseling
palace
acceptable
satisfied
glad
wins
measurements
verify
globe
trusted
copper
milwaukee
rack
medication
warehouse
shareware
dicke
kerry
receipt
supposed
ordinary
nobody
ghost
violation
configure
stability
applying
southwest
boss
pride
institutional
expectations
independence
knowing
reporter
metabolism
keith
champion
cloudy
linda
ross
personally
chile
anna
plenty
solo
sentence
throat
ignore
maria
uniform
excellence
wealth
tall
somewhere
vacuum
dancing
attributes
recognize
brass
writes
plaza
pdas
outcomes
survival
quest
publish
screening
thumbnail
trans
jonathan
whenever
nova
lifetime
pioneer
booty
forgotten
acrobat
plates
acres
venue
athletic
thermal
essays
behaviour
vital
telling
fairly
coastal
config
charity
intelligent
edinburgh
excel
modes
obligation
campbell
wake
stupid
harbor
hungary
traveler
segment
realize
regardless
enemy
puzzle
rising
aluminum
wells
wishlist
opens
insight
restricted
republican
secrets
lucky
latter
merchants
thick
trailers
repeat
syndrome
philips
attendance
penalty
drum
glasses
enables
iraqi
builder
vista
jessica
chips
terry
flood
foto
ease
arguments
amsterdam
arena
adventures
pupils
stewart
announcement
tabs
outcome
appreciate
expanded
casual
grown
polish
lovely
extras
centres
jerry
clause
smile
lands
troops
indoor
bulgaria
armed
broker
charger
regularly
believed
pine
cooling
tend
gulf
rick
trucks
mechanisms
divorce
laura
shopper
tokyo
partly
nikon
customize
tradition
candy
pills
tiger
donald
folks
sensor
exposed
telecom
hunt
angels
deputy
indicators
sealed
thai
emissions
physicians
loaded
fred
complaint
scenes
experiments
afghanistan
boost
spanking
scholarship
governance
mill
founded
supplements
chronic
icons
moral
catering
finger
keeps
pound
locate
camcorder
trained
burn
implementing
roses
labs
ourselves
bread
tobacco
wooden
motors
tough
roberts
incident
gonna
dynamics
conversation
decrease
chest
pension
billy
revenues
emerging
worship
capability
craig
herself
producing
churches
precision
damages
reserves
contributed
solve
shorts
reproduction
minority
diverse
ingredients
johnny
sole
franchise
recorder
complaints
facing
nancy
promotions
tones
passion
rehabilitation
maintaining
sight
laid
clay
defence
patches
weak
refund
towns
environments
trembl
divided
blvd
reception
wise
emails
cyprus
odds
correctly
insider
seminars
consequences
makers
hearts
geography
appearing
integrity
worry
discrimination
carter
legacy
marc
pleased
danger
vitamin
widely
processed
phrase
genuine
raising
implications
functionality
paradise
hybrid
reads
roles
intermediate
emotional
sons
leaf
glory
platforms
bigger
billing
diesel
versus
combine
overnight
geographic
exceed
saudi
fault
cuba
preliminary
districts
introduce
silk
promotional
kate
chevrolet
babies
karen
compiled
romantic
revealed
specialists
generator
albert
examine
jimmy
graham
suspension
bristol
margaret
compaq
correction
wolf
slowly
authentication
communicate
rugby
supplement
showtimes
portions
infant
promoting
sectors
samuel
fluid
grounds
fits
kick
regards
meal
hurt
machinery
bandwidth
unlike
equation
baskets
probability
dimension
wright
barry
proven
schedules
admissions
cached
warren
slip
studied
reviewer
involves
quarterly
profits
devil
grass
comply
marie
florist
illustrated
cherry
continental
alternate
deutsch
achievement
limitations
kenya
webcam
cuts
funeral
nutten
earrings
enjoyed
automated
chapters
charlie
quebec
passenger
convenient
dennis
mars
francis
sized
manga
noticed
socket
silent
literary
signals
caps
orientation
pill
theft
childhood
swing
symbols
meta
humans
analog
facial
choosing
talent
dated
flexibility
seeker
wisdom
shoot
boundary
mint
packard
offset
payday
philip
elite
spin
holders
believes
swedish
poems
deadline
jurisdiction
robot
displaying
witness
collins
equipped
stages
encouraged
winds
powder
broadway
acquired
assess
wash
cartridges
stones
entrance
gnome
roots
declaration
losing
attempts
gadgets
noble
glasgow
automation
impacts
gospel
advantages
shore
loves
induced
knight
preparing
loose
aims
recipient
linking
extensions
appeals
earned
illness
islamic
athletics
southeast
ieee
alternatives
pending
parker
determining
lebanon
corp
personalized
kennedy
conditioning
teenage
soap
triple
cooper
vincent
secured
unusual
answered
partnerships
destruction
slots
increasingly
migration
disorder
routine
toolbar
basically
rocks
conventional
titans
applicants
wearing
axis
sought
genes
mounted
habitat
firewall
median
guns
scanner
herein
occupational
animated
judicial
adjustment
hero
integer
treatments
bachelor
attitude
camcorders
engaged
falling
basics
montreal
carpet
struct
lenses
binary
genetics
attended
difficulty
punk
collective
coalition
dropped
enrollment
duke
walter
pace
besides
wage
producers
collector
hosts
interfaces
advertisers
moments
atlas
strings
dawn
representing
observation
feels
torture
carl
deleted
coat
mitchell
rica
restoration
convenience
returning
ralph
opposition
container
defendant
warner
confirmation
embedded
inkjet
supervisor
wizard
corps
actors
liver
peripherals
liable
brochure
morris
bestsellers
petition
eminem
recall
antenna
picked
assumed
departure
minneapolis
belief
killing
bikini
memphis
shoulder
decor
lookup
texts
harvard
brokers
diameter
ottawa
doll
podcast
seasons
peru
interactions
refine
bidder
singer
evans
herald
literacy
fails
aging
nike
intervention
plugin
attraction
diving
invite
modification
alice
latinas
suppose
customized
reed
involve
moderate
terror
younger
thirty
mice
opposite
understood
rapidly
dealtime
temp
intro
mercedes
assurance
clerk
happening
vast
mills
outline
amendments
tramadol
holland
receives
jeans
metropolitan
compilation
verification
fonts
wrap
refers
mood
favor
veterans
quiz
sigma
attractive
xhtml
occasion
recordings
jefferson
victim
demands
sleeping
careful
beam
gardening
obligations
arrive
orchestra
sunset
tracked
moreover
minimal
polyphonic
lottery
tops
framed
aside
outsourcing
licence
adjustable
allocation
michelle
essay
discipline
demonstrated
dialogue
identifying
alphabetical
camps
declared
dispatched
aaron
handheld
trace
disposal
shut
florists
packs
installing
switches
romania
voluntary
ncaa
thou
consult
greatly
blogging
mask
cycling
midnight
commonly
photographer
inform
turkish
coal
messaging
pentium
quantum
murray
intent
largely
pleasant
announce
constructed
additions
requiring
spoke
arrow
engagement
sampling
rough
weird
refinance
lion
inspired
holes
weddings
blade
suddenly
oxygen
cookie
meals
canyon
goto
meters
merely
calendars
arrangement
conclusions
passes
bibliography
pointer
compatibility
stretch
durham
furthermore
permits
cooperative
muslim
neil
sleeve
netscape
cleaner
cricket
beef
feeding
stroke
township
rankings
measuring
hats
robin
robinson
jacksonville
strap
headquarters
sharon
crowd
transfers
surf
olympic
transformation
remained
attachments
entities
customs
administrators
personality
rainbow
hook
roulette
decline
gloves
israeli
medicare
cord
skiing
cloud
facilitate
subscriber
valve
hewlett
explains
proceed
flickr
feelings
knife
jamaica
priorities
shelf
bookstore
timing
liked
parenting
adopt
denied
fotos
incredible
britney
freeware
donation
outer
crop
deaths
rivers
commonwealth
pharmaceutical
manhattan
tales
katrina
workforce
islam
nodes
thumbs
seeds
cited
lite
targeted
organizational
skype
realized
twelve
founder
decade
gamecube
dispute
portuguese
tired
titten
adverse
everywhere
excerpt
steam
discharge
drinks
voices
acute
halloween
climbing
stood
sing
tons
perfume
carol
honest
albany
hazardous
restore
stack
methodology
somebody
housewares
reputation
resistant
democrats
recycling
hang
curve
creator
amber
qualifications
museums
coding
slideshow
tracker
variation
passage
transferred
trunk
hiking
pierre
jelsoft
headset
photograph
oakland
colombia
waves
camel
distributor
lamps
underlying
hood
wrestling
suicide
archived
photoshop
arabia
gathering
projection
juice
chase
mathematical
logical
sauce
fame
extract
specialized
diagnostic
panama
indianapolis
payable
corporations
courtesy
criticism
automobile
confidential
statutory
accommodations
athens
northeast
downloaded
judges
retired
remarks
detected
decades
paintings
walked
arising
nissan
bracelet
eggs
juvenile
injection
yorkshire
populations
protective
afraid
acoustic
railway
cassette
initially
indicator
pointed
causing
mistake
norton
locked
eliminate
fusion
mineral
sunglasses
ruby
steering
beads
fortune
preference
canvas
threshold
parish
claimed
screens
cemetery
planner
croatia
flows
stadium
venezuela
exploration
mins
fewer
sequences
coupon
nurses
stem
proxy
astronomy
lanka
edwards
drew
contests
translate
announces
costume
tagged
berkeley
voted
killer
bikes
gates
adjusted
tune
bishop
pulled
corn
shaped
compression
seasonal
establishing
farmer
counters
puts
constitutional
grew
perfectly
slave
instantly
cultures
norfolk
coaching
examined
trek
encoding
litigation
submissions
heroes
painted
lycos
zdnet
broadcasting
horizontal
artwork
cosmetic
resulted
portrait
terrorist
informational
ethical
carriers
ecommerce
mobility
floral
builders
ties
struggle
schemes
suffering
neutral
fisher
spears
prospective
bedding
ultimately
joining
heading
equally
artificial
bearing
spectacular
coordination
connector
brad
combo
seniors
worlds
guilty
affiliated
activation
naturally
haven
tablet
jury
tail
subscribers
charm
lawn
violent
mitsubishi
underwear
basin
soup
potentially
ranch
constraints
crossing
inclusive
dimensional
cottage
drunk
considerable
crimes
resolved
mozilla
byte
toner
nose
latex
branches
anymore
oclc
delhi
holdings
alien
locator
selecting
processors
pantyhose
broke
nepal
zimbabwe
difficulties
juan
complexity
constantly
browsing
resolve
barcelona
presidential
documentary
territories
melissa
moscow
thesis
thru
jews
nylon
palestinian
discs
rocky
bargains
frequent
trim
nigeria
ceiling
pixels
ensuring
hispanic
legislature
hospitality
anybody
procurement
diamonds
espn
fleet
untitled
bunch
totals
marriott
singing
theoretical
afford
exercises
starring
referral
surveillance
optimal
quit
distinct
protocols
lung
highlight
substitute
inclusion
hopefully
brilliant
turner
sucking
cents
reuters
todd
spoken
omega
evaluated
stayed
civic
assignments
manuals
doug
sees
termination
watched
saver
thereof
grill
households
redeem
rogers
grain
authentic
regime
wanna
wishes
bull
montgomery
architectural
louisville
depend
differ
macintosh
movements
ranging
monica
repairs
breath
amenities
virtually
cole
mart
candle
hanging
colored
authorization
tale
verified
lynn
formerly
projector
situated
comparative
seeks
herbal
loving
strictly
routing
docs
stanley
psychological
surprised
retailer
vitamins
elegant
gains
renewal
genealogy
opposed
deemed
scoring
expenditure
brooklyn
liverpool
sisters
critics
connectivity
spots
algorithms
hacker
madrid
similarly
margin
coin
solely
fake
salon
collaborative
norman
excluding
turbo
headed
voters
cure
madonna
commander
arch
murphy
thinks
thats
suggestion
hdtv
soldier
phillips
asin
aimed
justin
bomb
harm
interval
mirrors
spotlight
tricks
reset
brush
investigate
expansys
panels
repeated
assault
connecting
spare
logistics
deer
kodak
tongue
bowling
danish
monkey
proportion
filename
skirt
florence
invest
honey
analyses
drawings
significance
scenario
lovers
atomic
approx
symposium
arabic
gauge
essentials
junction
protecting
faced
rachel
solving
transmitted
weekends
screenshots
produces
oven
intensive
chains
kingston
sixth
engage
deviant
noon
switching
quoted
adapters
correspondence
farms
imports
supervision
cheat
bronze
expenditures
sandy
separation
testimony
suspect
celebrities
macro
sender
mandatory
boundaries
crucial
syndication
celebration
adjacent
filtering
tuition
spouse
exotic
viewer
signup
threats
luxembourg
puzzles
reaching
damaged
cams
receptor
laugh
joel
surgical
destroy
citation
pitch
autos
premises
perry
proved
offensive
imperial
dozen
benjamin
deployment
teeth
cloth
studying
colleagues
stamp
lotus
salmon
olympus
separated
proc
cargo
directive
salem
mate
starter
upgrades
likes
butter
pepper
weapon
luggage
burden
chef
tapes
zones
races
isle
stylish
slim
maple
luke
grocery
offshore
governing
retailers
depot
kenneth
comp
blend
harrison
julie
occasionally
attending
emission
pete
spec
finest
realty
janet
penn
recruiting
apparent
instructional
phpbb
autumn
traveling
probe
midi
permissions
biotechnology
toilet
ranked
jackets
routes
packed
excited
outreach
helen
mounting
recover
tied
lopez
balanced
prescribed
catherine
timely
talked
debug
delayed
chuck
reproduced
dale
explicit
calculation
villas
ebook
consolidated
exclude
peeing
occasions
brooks
equations
newton
oils
sept
exceptional
anxiety
bingo
whilst
spatial
respondents
unto
ceramic
prompt
precious
minds
annually
considerations
scanners
xanax
pays
fingers
sunny
ebooks
delivers
queensland
necklace
musicians
leeds
composite
unavailable
cedar
arranged
lang
theaters
advocacy
raleigh
stud
fold
essentially
designing
threaded
qualify
blair
hopes
assessments
mason
diagram
burns
pumps
footwear
beijing
peoples
victor
mario
attach
licenses
utils
removing
advised
brunswick
spider
phys
ranges
pairs
sensitivity
trails
preservation
hudson
isolated
calgary
interim
assisted
divine
streaming
approve
chose
compound
intensity
technological
syndicate
abortion
dialog
venues
blast
wellness
calcium
newport
antivirus
addressing
pole
discounted
indians
shield
harvest
membrane
prague
previews
bangladesh
constitute
locally
concluded
pickup
desperate
mothers
nascar
iceland
demonstration
governmental
manufactured
candles
graduation
mega
bend
sailing
variations
moms
sacred
addiction
morocco
chrome
tommy
springfield
refused
brake
exterior
greeting
ecology
oliver
congo
glen
botswana
delays
synthesis
olive
undefined
unemployment
cyber
verizon
scored
enhancement
newcastle
clone
velocity
lambda
relay
composed
tears
performances
oasis
baseline
angry
societies
silicon
brazilian
identical
petroleum
compete
norwegian
lover
belong
honolulu
beatles
lips
retention
exchanges
pond
rolls
thomson
barnes
soundtrack
wondering
malta
daddy
ferry
rabbit
profession
seating
separately
physiology
collecting
exports
omaha
tire
participant
scholarships
recreational
dominican
chad
electron
loads
friendship
heather
passport
motel
unions
treasury
warrant
solaris
frozen
occupied
josh
royalty
scales
rally
observer
sunshine
strain
drag
ceremony
somehow
arrested
expanding
provincial
investigations
ripe
yamaha
rely
medications
hebrew
gained
rochester
dying
laundry
stuck
solomon
placing
stops
homework
adjust
assessed
advertiser
enabling
encryption
filling
downloadable
sophisticated
imposed
silence
scsi
focuses
soviet
possession
laboratories
treaty
vocal
trainer
organ
stronger
volumes
advances
vegetables
lemon
toxic
thumbnails
darkness
nuts
nail
bizrate
vienna
implied
span
stanford
stockings
joke
respondent
packing
statute
rejected
satisfy
destroyed
shelter
chapel
gamespot
manufacture
layers
wordpress
guided
vulnerability
accountability
celebrate
accredited
appliance
compressed
bahamas
powell
mixture
bench
univ
rider
scheduling
radius
perspectives
mortality
logging
hampton
christians
borders
therapeutic
pads
butts
inns
bobby
impressive
sheep
accordingly
architect
railroad
lectures
challenging
wines
nursery
harder
cups
microwave
cheapest
accidents
travesti
relocation
stuart
contributors
salvador
salad
monroe
tender
violations
foam
temperatures
paste
clouds
competitions
discretion
tanzania
preserve
poem
unsigned
staying
cosmetics
easter
theories
repository
praise
jeremy
venice
concentrations
estonia
christianity
veteran
streams
landing
signing
executed
katie
negotiations
realistic
showcase
integral
asks
relax
namibia
generating
christina
congressional
synopsis
hardly
prairie
reunion
composer
bean
sword
absent
photographic
sells
ecuador
hoping
accessed
spirits
modifications
coral
pixel
float
colin
bias
imported
paths
bubble
acquire
contrary
millennium
tribune
vessel
acids
focusing
viruses
cheaper
admitted
dairy
admit
fancy
equality
samoa
achieving
stickers
fisheries
exceptions
reactions
leasing
lauren
beliefs
macromedia
companion
squad
analyze
ashley
scroll
relate
divisions
swim
wages
additionally
suffer
forests
fellowship
nano
invalid
concerts
martial
males
victorian
retain
colours
execute
tunnel
genres
cambodia
patents
copyrights
chaos
lithuania
mastercard
wheat
chronicles
obtaining
beaver
updating
distribute
readings
decorative
kijiji
confused
compiler
enlargement
eagles
bases
accused
campaigns
unity
loud
conjunction
bride
rats
defines
airports
instances
indigenous
begun
brunette
packets
anchor
socks
validation
parade
corruption
stat
trigger
incentives
cholesterol
gathered
essex
slovenia
notified
differential
beaches
folders
dramatic
surfaces
terrible
routers
cruz
pendant
dresses
baptist
scientist
starsmerchant
hiring
clocks
arthritis
bios
females
wallace
nevertheless
reflects
taxation
fever
cuisine
surely
practitioners
transcript
myspace
theorem
inflation
thee
ruth
pray
stylus
compounds
pope
drums
contracting
arnold
structured
reasonably
jeep
chicks
bare
hung
cattle
radical
graduates
rover
recommends
controlling
treasure
reload
distributors
flame
levitra
tanks
assuming
monetary
elderly
arlington
mono
particles
floating
extraordinary
tile
indicating
bolivia
spell
hottest
stevens
coordinate
kuwait
exclusively
emily
alleged
limitation
widescreen
compile
webster
struck
illustration
plymouth
warnings
construct
apps
inquiries
bridal
annex
inspiration
tribal
curious
affecting
freight
rebate
meetup
eclipse
sudan
downloading
shuttle
aggregate
stunning
cycles
affects
forecasts
detect
actively
ciao
ampland
knee
prep
complicated
chem
fastest
butler
shopzilla
injured
decorating
payroll
cookbook
expressions
courier
uploaded
shakespeare
hints
collapse
americas
connectors
unlikely
pros
conflicts
techno
beverage
tribute
wired
elvis
immune
latvia
travelers
forestry
barriers
cant
rarely
infected
offerings
martha
genesis
barrier
argue
incorrect
trains
metals
bicycle
furnishings
letting
arise
guatemala
celtic
thereby
jamie
particle
perception
minerals
advise
humidity
bottles
boxing
bangkok
renaissance
pathology
sara
ordinance
hughes
photographers
infections
jeffrey
chess
operates
brisbane
configured
survive
oscar
festivals
menus
joan
possibilities
duck
reveal
canal
amino
contributing
herbs
clinics
manitoba
analytical
missions
watson
lying
costumes
strict
dive
saddam
circulation
drill
offense
bryan
protest
assumption
jerusalem
hobby
tries
transexuales
invention
nickname
fiji
technician
inline
executives
enquiries
washing
audi
staffing
cognitive
exploring
trick
enquiry
closure
raid
timber
volt
intense
playlist
registrar
showers
supporters
ruling
steady
dirt
statutes
withdrawal
myers
drops
predicted
wider
saskatchewan
cancellation
plugins
enrolled
sensors
screw
ministers
publicly
hourly
blame
geneva
freebsd
veterinary
acer
prostores
reseller
dist
handed
suffered
intake
informal
relevance
incentive
butterfly
tucson
mechanics
heavily
swingers
fifty
headers
mistakes
numerical
geek
uncle
defining
counting
reflection
sink
accompanied
assure
invitation
devoted
princeton
jacob
sodium
randy
spirituality
hormone
meanwhile
proprietary
timothy
childrens
brick
grip
naval
thumbzilla
medieval
porcelain
bridges
pichunter
captured
watt
thehun
decent
casting
dayton
translated
shortly
cameron
columnists
pins
carlos
reno
donna
andreas
warrior
diploma
cabin
innocent
scanning
consensus
polo
valium
copying
delivering
cordless
patricia
horn
eddie
uganda
fired
journalism
prot
trivia
adidas
perth
frog
grammar
intention
syria
disagree
klein
harvey
tires
logs
undertaken
hazard
retro
statewide
semiconductor
gregory
episodes
boolean
circular
anger
mainland
illustrations
suits
chances
interact
snap
happiness
substantially
bizarre
glenn
auckland
olympics
fruits
identifier
ribbon
calculations
jpeg
conducting
startup
suzuki
trinidad
kissing
handy
swap
exempt
crops
reduces
accomplished
calculators
geometry
impression
slovakia
flip
guild
correlation
gorgeous
capitol
dishes
barbados
chrysler
nervous
refuse
extends
fragrance
mcdonald
replica
plumbing
brussels
tribe
neighbors
trades
superb
buzz
transparent
nuke
trinity
charleston
handled
legends
boom
calm
champions
floors
selections
projectors
inappropriate
exhaust
comparing
shanghai
speaks
burton
vocational
davidson
copied
scotia
farming
gibson
pharmacies
fork
troy
roller
introducing
batch
organize
appreciated
alter
nicole
latino
ghana
edges
mixing
handles
skilled
fitted
albuquerque
harmony
distinguished
asthma
projected
assumptions
shareholders
twins
developmental
zope
regulated
triangle
amend
anticipated
oriental
reward
windsor
zambia
completing
gmbh
hydrogen
webshots
sprint
comparable
chick
advocate
sims
confusion
copyrighted
tray
inputs
warranties
genome
escorts
documented
thong
medal
paperbacks
coaches
vessels
harbour
walks
keyboards
sage
knives
vulnerable
arrange
artistic
honors
booth
indie
reflected
unified
bones
breed
detector
ignored
polar
fallen
precise
sussex
respiratory
notifications
msgid
transexual
mainstream
invoice
evaluating
subcommittee
gather
suse
maternity
backed
alfred
colonial
carey
motels
forming
embassy
cave
journalists
danny
rebecca
slight
proceeds
indirect
amongst
wool
foundations
msgstr
arrest
volleyball
adipex
horizon
deeply
toolbox
marina
liabilities
prizes
bosnia
browsers
decreased
patio
tolerance
surfing
creativity
lloyd
describing
optics
pursue
lightning
overcome
eyed
quotations
grab
inspector
attract
brighton
beans
bookmarks
ellis
disable
snake
succeed
leonard
lending
oops
reminder
searched
behavioral
riverside
bathrooms
plains
raymond
insights
abilities
initiated
sullivan
midwest
karaoke
trap
lonely
fool
nonprofit
lancaster
suspended
hereby
observe
julia
containers
attitudes
karl
berry
collar
simultaneously
racial
integrate
bermuda
amanda
sociology
mobiles
screenshot
exhibitions
kelkoo
confident
retrieved
exhibits
officially
consortium
dies
terrace
bacteria
replied
seafood
novels
recipients
ought
delicious
traditions
jail
safely
finite
kidney
periodically
fixes
sends
durable
mazda
allied
throws
moisture
hungarian
roster
referring
symantec
spencer
wichita
nasdaq
uruguay
transform
timer
tablets
tuning
gotten
educators
tyler
futures
vegetable
verse
highs
humanities
independently
wanting
custody
scratch
launches
ipaq
alignment
henderson
britannica
comm
ellen
competitors
rocket
bullet
towers
racks
lace
nasty
visibility
latitude
consciousness
tumor
ugly
deposits
beverly
mistress
encounter
trustees
watts
duncan
reprints
hart
bernard
resolutions
ment
accessing
forty
tubes
attempted
midlands
priest
floyd
ronald
analysts
queue
trance
locale
nicholas
biol
bundle
hammer
invasion
witnesses
runner
rows
administered
notion
skins
mailed
fujitsu
spelling
arctic
exams
rewards
beneath
strengthen
defend
frederick
medicaid
treo
infrared
seventh
gods
welsh
belly
aggressive
advertisements
quarters
stolen
soonest
haiti
disturbed
determines
sculpture
poly
ears
fist
naturals
motivation
lenders
pharmacology
fitting
fixtures
bloggers
mere
agrees
passengers
quantities
petersburg
consistently
powerpoint
cons
surplus
elder
sonic
obituaries
cheers
taxi
punishment
appreciation
subsequently
belarus
zoning
gravity
providence
thumb
restriction
incorporate
backgrounds
treasurer
guitars
essence
flooring
lightweight
ethiopia
mighty
athletes
humanity
transcription
holmes
complications
scholars
scripting
remembered
galaxy
chester
snapshot
caring
worn
synthetic
shaw
segments
testament
expo
dominant
twist
specifics
itunes
stomach
partially
buried
newbie
minimize
darwin
ranks
wilderness
debut
generations
tournaments
bradley
deny
anatomy
bali
judy
sponsorship
headphones
fraction
trio
proceeding
cube
defects
volkswagen
uncertainty
breakdown
milton
marker
reconstruction
subsidiary
strengths
clarity
rugs
sandra
adelaide
encouraging
furnished
monaco
settled
folding
emirates
terrorists
airfare
comparisons
beneficial
distributions
vaccine
belize
fate
viewpicture
promised
volvo
penny
robust
bookings
threatened
minolta
republicans
discusses
porter
gras
jungle
responded
abstracts
ivory
alpine
prediction
pharmaceuticals
andale
fabulous
remix
alias
thesaurus
individually
battlefield
literally
newer
ecological
spice
oval
implies
soma
cooler
appraisal
consisting
maritime
periodic
submitting
overhead
ascii
prospect
shipment
breeding
citations
geographical
donor
mozambique
tension
href
benz
trash
shapes
wifi
tier
earl
manor
envelope
diane
homeland
disclaimers
championships
excluded
andrea
breeds
rapids
disco
sheffield
bailey
endif
finishing
emotions
wellington
incoming
prospects
lexmark
cleaners
bulgarian
eternal
cashiers
guam
cite
aboriginal
remarkable
rotation
preventing
productive
boulevard
eugene
metric
compliant
minus
penalties
bennett
imagination
hotmail
refurbished
joshua
armenia
varied
grande
closest
activated
actress
mess
conferencing
assign
armstrong
politicians
trackbacks
accommodate
tigers
aurora
slides
milan
premiere
lender
villages
shade
chorus
christine
rhythm
digit
argued
dietary
symphony
clarke
sudden
accepting
precipitation
marilyn
lions
findlaw
pools
lyric
claire
isolation
speeds
sustained
matched
approximate
rope
carroll
rational
programmer
fighters
chambers
dump
greetings
inherited
warming
incomplete
vocals
chronicle
fountain
chubby
grave
legitimate
biographies
burner
investigator
plaintiff
finnish
gentle
prisoners
deeper
muslims
hose
mediterranean
nightlife
footage
howto
worthy
reveals
architects
saints
entrepreneur
carries
freelance
excessive
devon
screensaver
helena
saves
regarded
valuation
unexpected
cigarette
characteristic
marion
lobby
egyptian
tunisia
metallica
outlined
consequently
headline
treating
punch
appointments
gotta
cowboy
narrative
bahrain
enormous
karma
consist
betty
queens
academics
pubs
quantitative
lucas
screensavers
subdivision
tribes
defeat
clicks
distinction
honduras
naughty
hazards
insured
harper
livestock
mardi
exemption
tenant
sustainability
cabinets
tattoo
shake
algebra
shadows
holly
formatting
silly
nutritional
mercy
hartford
freely
marcus
sunrise
wrapping
mild
nicaragua
weblogs
timeline
belongs
readily
affiliation
fence
nudist
infinite
diana
ensures
relatives
lindsay
clan
legally
shame
satisfactory
revolutionary
bracelets
sync
civilian
telephony
mesa
fatal
remedy
realtors
breathing
briefly
thickness
adjustments
graphical
genius
discussing
aerospace
fighter
meaningful
flesh
retreat
adapted
barely
wherever
estates
democrat
borough
maintains
failing
shortcuts
retained
voyeurweb
pamela
andrews
marble
extending
jesse
specifies
hull
logitech
surrey
briefing
belkin
accreditation
blackberry
highland
meditation
modular
microphone
macedonia
combining
brandon
instrumental
giants
organizing
shed
balloon
moderators
winston
memo
solved
tide
kazakhstan
hawaiian
standings
partition
invisible
gratuit
consoles
funk
qatar
magnet
translations
porsche
cayman
jaguar
reel
sheer
commodity
posing
kilometers
bind
thanksgiving
rand
hopkins
urgent
guarantees
infants
gothic
cylinder
witch
buck
indication
congratulations
cohen
usgs
puppy
kathy
acre
graphs
surround
cigarettes
revenge
expires
enemies
lows
controllers
aqua
chen
emma
consultancy
finances
accepts
enjoying
conventions
patrol
smell
pest
italiano
coordinates
carnival
roughly
sticker
promises
responding
reef
physically
divide
stakeholders
hydrocodone
consecutive
cornell
satin
deserve
attempting
mailto
promo
representations
chan
worried
tunes
garbage
competing
combines
beth
bradford
phrases
peninsula
chelsea
boring
reynolds
jill
accurately
speeches
reaches
schema
considers
sofa
catalogs
ministries
vacancies
quizzes
parliamentary
prefix
lucia
savannah
barrel
typing
nerve
dans
planets
deficit
boulder
pointing
renew
coupled
viii
myanmar
metadata
harold
circuits
floppy
texture
handbags
somerset
incurred
acknowledge
thoroughly
antigua
nottingham
thunder
tent
caution
identifies
questionnaire
qualification
locks
modelling
namely
miniature
dept
hack
dare
euros
interstate
pirates
aerial
hawk
consequence
rebel
systematic
perceived
origins
hired
makeup
textile
lamb
madagascar
nathan
tobago
presenting
troubleshooting
uzbekistan
indexes
centuries
magnitude
richardson
hindu
fragrances
vocabulary
licking
earthquake
fundraising
markers
weights
albania
geological
assessing
lasting
wicked
introduces
kills
roommate
webcams
pushed
webmasters
computational
acdbentity
participated
junk
handhelds
lucy
answering
hans
impressed
slope
reggae
failures
poet
conspiracy
surname
theology
nails
evident
whats
rides
rehab
epic
saturn
organizer
allergy
sake
twisted
combinations
preceding
merit
enzyme
cumulative
zshops
planes
edmonton
tackle
disks
condo
pokemon
amplifier
ambien
arbitrary
prominent
retrieve
lexington
vernon
sans
worldcat
titanium
fairy
builds
contacted
shaft
lean
recorders
occasional
leslie
casio
deutsche
postings
innovations
kitty
postcards
dude
drain
monte
fires
algeria
blessed
luis
reviewing
cardiff
cornwall
favors
potato
panic
explicitly
sticks
leone
transsexual
citizenship
excuse
reforms
basement
onion
strand
sandwich
lawsuit
alto
informative
girlfriend
bloomberg
cheque
hierarchy
influenced
banners
reject
abandoned
circles
italic
beats
merry
scuba
gore
complement
cult
dash
passive
mauritius
valued
cage
checklist
requesting
courage
verde
lauderdale
scenarios
gazette
hitachi
divx
extraction
batman
elevation
hearings
coleman
hugh
utilization
beverages
calibration
jake
eval
efficiently
anaheim
ping
textbook
dried
entertaining
prerequisite
luther
frontier
settle
stopping
refugees
knights
hypothesis
palmer
medicines
flux
derby
peaceful
altered
pontiac
regression
doctrine
scenic
trainers
muze
enhancements
renewable
intersection
passwords
sewing
consistency
collectors
conclude
recognised
munich
oman
celebs
propose
azerbaijan
lighter
rage
adsl
prix
astrology
advisors
pavilion
tactics
trusts
occurring
supplemental
travelling
talented
annie
pillow
induction
derek
precisely
shorter
harley
spreading
provinces
relying
finals
paraguay
steal
parcel
refined
fifteen
widespread
incidence
fears
predict
boutique
acrylic
rolled
tuner
avon
incidents
peterson
rays
shannon
toddler
enhancing
flavor
alike
walt
homeless
horrible
hungry
metallic
acne
blocked
interference
warriors
palestine
listprice
libs
undo
cadillac
atmospheric
malawi
sagem
knowledgestorm
dana
halo
curtis
parental
referenced
strikes
lesser
publicity
marathon
proposition
gays
pressing
gasoline
dressed
scout
belfast
exec
dealt
niagara
warcraft
charms
catalyst
trader
bucks
allowance
denial
designation
thrown
prepaid
raises
duplicate
electro
criterion
badge
wrist
civilization
analyzed
vietnamese
heath
tremendous
ballot
lexus
varying
remedies
validity
trustee
maui
weighted
angola
performs
plastics
realm
corrected
jenny
helmet
salaries
postcard
elephant
yemen
encountered
tsunami
scholar
nickel
internationally
surrounded
buses
expedia
geology
creatures
coating
commented
wallet
cleared
smilies
vids
accomplish
boating
drainage
shakira
corners
broader
vegetarian
rouge
yeast
yale
newfoundland
clearing
investigated
ambassador
coated
intend
stephanie
contacting
vegetation
doom
findarticles
louise
kenny
specially
owen
routines
hitting
yukon
beings
bite
issn
aquatic
reliance
habits
striking
myth
infectious
podcasts
singh
gilbert
ferrari
continuity
brook
outputs
phenomenon
ensemble
insulin
assured
biblical
weed
conscious
accent
mysimon
eleven
wives
ambient
utilize
mileage
oecd
prostate
adaptor
auburn
unlock
hyundai
pledge
vampire
angela
relates
nitrogen
xerox
dice
merger
softball
referrals
quad
dock
differently
firewire
mods
nextel
framing
organised
musician
blocking
rwanda
sorts
integrating
vsnet
limiting
dispatch
revisions
papua
restored
hint
armor
riders
chargers
remark
dozens
varies
msie
reasoning
rendered
picking
charitable
guards
annotated
convinced
openings
buys
burlington
replacing
researcher
watershed
councils
occupations
acknowledged
kruger
pockets
granny
pork
equilibrium
viral
inquire
pipes
characterized
laden
aruba
cottages
realtor
merge
privilege
edgar
develops
qualifying
chassis
dubai
estimation
barn
pushing
fleece
pediatric
fare
asus
pierce
allan
dressing
techrepublic
sperm
bald
filme
craps
fuji
frost
leon
institutes
mold
dame
sally
yacht
tracy
prefers
drilling
brochures
herb
alot
breach
whale
traveller
appropriations
suspected
tomatoes
benchmark
beginners
instructors
highlighted
bedford
stationery
idle
mustang
unauthorized
clusters
antibody
competent
momentum
wiring
pastor
calvin
shark
contributor
demonstrates
phases
grateful
emerald
gradually
laughing
grows
cliff
desirable
tract
ballet
journalist
abraham
bumper
afterwards
webpage
religions
garlic
hostels
shine
senegal
explosion
banned
wendy
briefs
signatures
diffs
cove
mumbai
ozone
disciplines
casa
daughters
conversations
radios
tariff
nvidia
opponent
pasta
simplified
muscles
serum
wrapped
swift
motherboard
runtime
inbox
focal
bibliographic
eden
distant
incl
champagne
decimal
deviation
superintendent
propecia
samba
hostel
housewives
employ
mongolia
penguin
magical
influences
inspections
irrigation
miracle
manually
reprint
reid
hydraulic
centered
robertson
flex
yearly
penetration
wound
belle
rosa
conviction
hash
omissions
writings
hamburg
lazy
retrieval
qualities
cindy
fathers
carb
charging
marvel
lined
prototype
importantly
petite
apparatus
terrain
pens
explaining
strips
gossip
rangers
nomination
empirical
rotary
worm
dependence
discrete
beginner
boxed
sexuality
polyester
cubic
deaf
commitments
suggesting
sapphire
kinase
skirts
mats
remainder
crawford
labeled
privileges
televisions
specializing
marking
commodities
serbia
sheriff
griffin
declined
guyana
spies
blah
mime
neighbor
motorcycles
elect
highways
thinkpad
concentrate
intimate
reproductive
preston
deadly
feof
bunny
chevy
molecules
rounds
longest
refrigerator
tions
intervals
sentences
dentists
usda
exclusion
workstation
holocaust
keen
flyer
peas
dosage
receivers
urls
customise
disposition
variance
navigator
investigators
cameroon
baking
marijuana
adaptive
computed
needle
baths
cathedral
brakes
nirvana
fairfield
owns
invision
sticky
destiny
generous
madness
emacs
climb
blowing
fascinating
landscapes
heated
lafayette
jackie
computation
cardiovascular
sparc
cardiac
salvation
dover
adrian
predictions
accompanying
vatican
brutal
learners
selective
arbitration
configuring
token
editorials
zinc
sacrifice
seekers
guru
removable
convergence
yields
gibraltar
levy
suited
numeric
anthropology
skating
kinda
aberdeen
emperor
grad
malpractice
dylan
bras
belts
blacks
educated
rebates
reporters
burke
proudly
necessity
rendering
inserted
pulling
basename
kyle
obesity
curves
suburban
touring
clara
vertex
hepatitis
nationally
tomato
andorra
waterproof
expired
travels
flush
waiver
pale
specialties
hayes
humanitarian
invitations
functioning
delight
survivor
garcia
cingular
economies
alexandria
bacterial
moses
counted
undertake
declare
continuously
johns
valves
gaps
impaired
achievements
donors
tear
jewel
teddy
convertible
teaches
ventures
bufing
stranger
tragedy
julian
nest
dryer
painful
velvet
tribunal
ruled
nato
pensions
prayers
funky
secretariat
nowhere
paragraphs
gale
joins
adolescent
nominations
wesley
lately
cancelled
scary
mattress
mpegs
brunei
likewise
banana
introductory
slovak
cakes
stan
reservoir
occurrence
idol
mixer
remind
worcester
sbjct
demographic
charming
tooth
disciplinary
annoying
respected
stays
disclose
affair
drove
washer
upset
restrict
springer
beside
mines
portraits
rebound
logan
mentor
interpreted
evaluations
fought
baghdad
elimination
metres
hypothetical
immigrants
complimentary
helicopter
pencil
freeze
performer
titled
commissions
sphere
powerseller
moss
ratios
concord
graduated
endorsed
surprising
walnut
lance
ladder
italia
unnecessary
dramatically
liberia
sherman
cork
maximize
hansen
senators
workout
mali
yugoslavia
bleeding
characterization
colon
likelihood
lanes
purse
fundamentals
contamination
endangered
compromise
optimize
stating
dome
caroline
expiration
namespace
align
peripheral
bless
engaging
negotiation
crest
opponents
triumph
nominated
confidentiality
electoral
changelog
welding
deferred
alternatively
heel
alloy
condos
plots
polished
yang
gently
greensboro
tulsa
locking
casey
controversial
draws
fridge
blanket
bloom
simpsons
elliott
recovered
fraser
justify
upgrading
blades
loops
surge
frontpage
trauma
tahoe
advert
possess
demanding
defensive
flashers
subaru
forbidden
vanilla
programmers
monitored
installations
deutschland
picnic
souls
arrivals
spank
practitioner
motivated
dumb
smithsonian
hollow
vault
securely
examining
fioricet
groove
revelation
pursuit
delegation
wires
dictionaries
mails
backing
greenhouse
sleeps
blake
transparency
travis
endless
figured
orbit
currencies
niger
bacon
survivors
positioning
heater
colony
cannon
circus
promoted
forbes
moldova
descending
paxil
spine
trout
enclosed
feat
temporarily
ntsc
cooked
thriller
transmit
apnic
fatty
gerald
pressed
frequencies
scanned
reflections
hunger
mariah
municipality
usps
joyce
detective
surgeon
cement
experiencing
fireplace
endorsement
planners
disputes
textiles
missile
intranet
closes
psychiatry
persistent
deborah
conf
marco
assists
summaries
glow
gabriel
auditor
aquarium
violin
prophet
bracket
looksmart
isaac
oxide
oaks
magnificent
erik
colleague
naples
promptly
modems
adaptation
harmful
paintball
prozac
sexually
enclosure
dividend
newark
paso
glucose
phantom
norm
playback
supervisors
westminster
turtle
distances
absorption
treasures
warned
neural
ware
fossil
hometown
badly
transcripts
apollo
disappointed
persian
continually
communist
collectible
handmade
greene
entrepreneurs
robots
grenada
creations
jade
scoop
acquisitions
foul
keno
earning
mailman
sanyo
nested
biodiversity
excitement
somalia
movers
verbal
blink
presently
seas
carlo
workflow
mysterious
novelty
bryant
tiles
voyuer
librarian
subsidiaries
switched
stockholm
tamil
garmin
pose
fuzzy
indonesian
grams
therapist
richards
mrna
budgets
toolkit
promising
relaxation
goat
render
carmen
thereafter
hardwood
erotica
temporal
sail
forge
commissioners
dense
brave
forwarding
awful
nightmare
airplane
reductions
southampton
istanbul
impose
organisms
sega
telescope
viewers
asbestos
portsmouth
cdna
meyer
enters
savage
advancement
harassment
willow
resumes
bolt
gage
throwing
existed
generators
wagon
barbie
favour
knock
urge
smtp
generates
potatoes
thorough
replication
inexpensive
kurt
receptors
peers
roland
optimum
neon
interventions
quilt
huntington
creature
ours
mounts
syracuse
internship
lone
refresh
aluminium
snowboard
beastality
webcast
michel
evanescence
subtle
coordinated
notre
shipments
maldives
stripes
firmware
antarctica
cope
shepherd
canberra
cradle
chancellor
mambo
lime
kirk
flour
controversy
legendary
bool
sympathy
choir
avoiding
beautifully
blond
expects
jumping
fabrics
antibodies
polymer
hygiene
poultry
virtue
burst
examinations
surgeons
bouquet
immunology
promotes
mandate
wiley
departmental
spas
corpus
johnston
terminology
gentleman
fibre
reproduce
convicted
shades
jets
indices
roommates
adware
intl
threatening
spokesman
zoloft
activists
frankfurt
prisoner
daisy
halifax
encourages
ultram
cursor
assembled
earliest
donated
stuffed
restructuring
insects
terminals
crude
morrison
maiden
simulations
sufficiently
examines
viking
myrtle
bored
cleanup
yarn
knit
conditional
crossword
bother
budapest
conceptual
knitting
attacked
bhutan
liechtenstein
mating
compute
redhead
arrives
translator
automobiles
tractor
allah
continent
unwrap
fares
longitude
resist
challenged
telecharger
hoped
pike
safer
insertion
instrumentation
hugo
wagner
constraint
groundwater
touched
strengthening
cologne
gzip
wishing
ranger
smallest
insulation
newman
marsh
ricky
ctrl
scared
theta
infringement
bent
laos
subjective
monsters
asylum
lightbox
robbie
stake
cocktail
outlets
swaziland
varieties
arbor
mediawiki
configurations
poison

Overwriting words.txt


In [20]:
import os
import asyncio
import time
import random
from typing import List, Tuple, Dict, Optional

# Import AutoGen classes
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

# Import core game utilities from scrabble_utils.py (must be in the same directory)
from L5_utils import ScrabbleState, AlphaBetaMinimax, MonteCarlo, Move

### First take a look at the Scrabble Implementation

The following code randomly selects moves for both players in the Scrabble game until the game terminates.
It should give you a sense of what the API looks like.

In [21]:
# Initialize the game state
game_state = ScrabbleState.create_new_game()
print("Initial Game State:")
print(game_state)

while not game_state.is_terminal():
    current_player = game_state.current_player
    legal_moves = game_state.get_legal_moves(current_player)

    # Prefer non-pass moves if available
    non_pass_moves = [move for move in legal_moves if not move.is_pass]
    if non_pass_moves:
        chosen_move = random.choice(non_pass_moves)
    else:
        chosen_move = legal_moves[0]  # Must pass if no other

    game_state = game_state.apply_move(chosen_move)
    print(f"\nPlayer {current_player} plays: {chosen_move}")
    print(game_state)

Initial Game State:
SCRABBLE GAME STATE

Current Player: Player 1
Scores: Player 1: 0, Player 2: 0
Consecutive Passes: 0

Player 1 Rack: O W A A Q A R (7 tiles)
Player 2 Rack: V S D N G R M (7 tiles)

Tiles Remaining in Pool: 86

Board:
Legend: · = empty, ² = 2x letter, ³ = 3x letter, * = 2x word, # = 3x word
     0  1  2  3  4  5  6  7  8  9 10 11 12 13 14
   +---------------------------------------------+
 0 | ·  ²  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ²  · |
 1 | ²  ·  ·  ·  ·  ³  ·  ·  ·  ³  ·  ·  ·  ·  ² |
 2 | ·  ·  *  ·  ·  ·  ²  ·  ²  ·  ·  ·  *  ·  · |
 3 | ·  ·  ·  #  ·  ·  ·  ²  ·  ·  ·  #  ·  ·  · |
 4 | ·  ·  ·  ·  *  ·  ·  ·  ·  ·  *  ·  ·  ·  · |
 5 | ·  ³  ·  ·  ·  ³  ·  ·  ·  ³  ·  ·  ·  ³  · |
 6 | ·  ·  ²  ·  ·  ·  ²  ·  ²  ·  ·  ·  ²  ·  · |
 7 | ·  ·  ·  ²  ·  ·  ·  *  ·  ·  ·  ²  ·  ·  · |
 8 | ·  ·  ²  ·  ·  ·  ²  ·  ²  ·  ·  ·  ²  ·  · |
 9 | ·  ³  ·  ·  ·  ³  ·  ·  ·  ³  ·  ·  ·  ³  · |
10 | ·  ·  ·  ·  *  ·  ·  ·  ·  ·  *  ·  ·  ·  · |
11 | ·  ·  ·  #  ·  ·  ·  ² 

### Azure OpenAI Configuration

In [22]:
from google.colab import userdata
# Reusing configuration from L4.ipynb
# Configure your Azure OpenAI client
azure_deployment = "gpt-5-mini-lab1"
api_version = "2024-12-01-preview"
azure_endpoint = "https://lab1agent-aifoundry.cognitiveservices.azure.com/"

# Ensure your API key is set as an environment variable
api_key = userdata.get('api_key')

client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_deployment,
    model="gpt-5-mini", # Using the default model name referenced in previous labs
    api_version=api_version,
    azure_endpoint=azure_endpoint,
    api_key=api_key,
)

---

## Part 1 & 2: Classical Search Limitations

A small note: Minimax and MCTS expect that the problem space is zero-sum. Scrabble is not inherently zero-sum with the typical method scores are calculated. However, we make it zero-sum by using the difference between the two players' scores as the metric to optimize for.

### 1. Written Analysis: Why Minimax Fails Scrabble (Part 2)

**Task:** Using the concepts of **State Space Size, Branching Factor, and Imperfect Information** (hidden opponent tiles), justify why a naive application of Minimax with Alpha-Beta pruning is impractical for Scrabble.

_Minimax (with Alpha-Beta pruning) is impractical for Scrabble for multiple
reasons. First, the Minimax algorithm requires the entire game tree which is
impractical for Scrabble, if not impossible. The state space is too big in
Scrabble, plus the Branching Factor is too vast with all the different letter
and word combinations. Additionally, you can't see the opponent's tiles so it
would be impossible to factor their moves into the tree._

### 2. Demonstration: Minimax Performance Limits

**Task:** Run the `AlphaBetaMinimax` on a simplified Scrabble state. Observe the exploration time and the number of nodes explored, particularly when increasing the `max_depth` to demonstrate the complexity explosion.

In [23]:
# Initialize a simple Scrabble state
initial_state = ScrabbleState.create_new_game()
print(f"Initial state created. Player 1 Rack: {initial_state.racks}")

# Experiment 1: Shallow Search (Depth 1)
print("\n--- Running Minimax (Depth 1) ---")
minimax_d1 = AlphaBetaMinimax(max_depth=1)
best_move_d1 = minimax_d1.find_best_move(initial_state)
print(f"Depth 1 suggests: {best_move_d1}")

# Experiment 2: Deeper Search (Depth 3)
print("\n--- Running Minimax (Depth 3) ---")
minimax_d3 = AlphaBetaMinimax(max_depth=3)

best_move_d3 = minimax_d3.find_best_move(initial_state)
print(f"Depth 3 suggests: {best_move_d3}")

Initial state created. Player 1 Rack: {1: ['A', 'N', 'V', 'N', 'O', 'F', 'E'], 2: ['G', 'Y', 'S', 'E', 'S', 'D', 'L']}

--- Running Minimax (Depth 1) ---
Minimax search completed in: 0.0019s. Nodes explored: 113. Best move value: 14.00
Depth 1 suggests: Move(Word='AVON', Score=14, Tile Positions=[(7, 4), (7, 5), (7, 6), (7, 7)])

--- Running Minimax (Depth 3) ---
Minimax search completed in: 23.4714s. Nodes explored: 1534. Best move value: 41.00
Depth 3 suggests: Move(Word='AVON', Score=14, Tile Positions=[(5, 7), (6, 7), (7, 7), (8, 7)])


---

## Part 3: Adversarial Monte Carlo Agent

Now check out MCTS with the placeholder heuristic (which currently uses the score + noise).

In [24]:
async def basic_scrabble_heuristic(state: ScrabbleState, player: int) -> float:
    # Simple heuristic: score + small random noise
    base_score = state.scores[player]
    noise = random.uniform(-0.1, 0.1)  # Small noise to break ties
    return base_score + noise


Here we're simulating a game between two players. One which uses MCTS to choose it's moves (Player 1) and the other which chooses at random (Player 2).

In [25]:
async def run_mcts_match(num_playouts: int, heuristic_fn, num_turns: int = 5):
    """Initializes and runs the Monte Carlo Agent against a simple opponent."""

    # MCTS Agent (Player 1) uses the given scrabble_heuristic
    mcts = MonteCarlo(num_playouts=num_playouts, heuristic_fn=heuristic_fn)

    current_state = ScrabbleState.create_new_game()

    print(
        f"Starting Scrabble game simulation for {num_turns} turns. MCTS Agent (P1) uses {int(num_playouts)} playouts."
    )

    for turn in range(num_turns):
        if current_state.is_terminal():
            break

        player_id = current_state.current_player

        if player_id == 1:
            # Player 1 uses MCTS
            print(f"\n--- Turn {turn+1}: Player 1 (MCTS) ---")
            move = await mcts.find_best_move(current_state)

        else:
            # Player 2 makes moves randomly (baseline)
            print(f"\n--- Turn {turn+1}: Player 2 (Random Baseline) ---")
            possible_moves = current_state.get_legal_moves(player_id)
            move = random.choice(possible_moves)

        current_state = current_state.apply_move(move)
        print(f"Player {player_id} plays: {move}")
        print(current_state)


# Task: Run a baseline MCTS match
await run_mcts_match(num_playouts=1e5, heuristic_fn=basic_scrabble_heuristic)

Starting Scrabble game simulation for 5 turns. MCTS Agent (P1) uses 100000 playouts.

--- Turn 1: Player 1 (MCTS) ---
Starting MCTS: 662 playouts per move (for 151 possible moves)...
Player 1 plays: Move(Word='NEARBY', Score=30, Tile Positions=[(6, 7), (7, 7), (8, 7), (9, 7), (10, 7), (11, 7)])
SCRABBLE GAME STATE

Current Player: Player 2
Scores: Player 1: 30, Player 2: 0
Consecutive Passes: 0

Player 1 Rack: I O S N O A E (7 tiles)
Player 2 Rack: A T R N _ O Z (7 tiles)

Tiles Remaining in Pool: 80

Board:
Legend: · = empty, ² = 2x letter, ³ = 3x letter, * = 2x word, # = 3x word
     0  1  2  3  4  5  6  7  8  9 10 11 12 13 14
   +---------------------------------------------+
 0 | ·  ²  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ²  · |
 1 | ²  ·  ·  ·  ·  ³  ·  ·  ·  ³  ·  ·  ·  ·  ² |
 2 | ·  ·  *  ·  ·  ·  ²  ·  ²  ·  ·  ·  *  ·  · |
 3 | ·  ·  ·  #  ·  ·  ·  ²  ·  ·  ·  #  ·  ·  · |
 4 | ·  ·  ·  ·  *  ·  ·  ·  ·  ·  *  ·  ·  ·  · |
 5 | ·  ³  ·  ·  ·  ³  ·  ·  ·  ³  ·  ·  ·  ³  · |
 6 | ·

---

## Part 4: Agentic Heuristic Design and Impact


Implement the `collaborative_heuristic` function using AutoGen agent collaboration.

Note: Like previous labs that use an agentic heuristic function, this will take a long time to run. An agentic system for this purpose is impractical due to the overhead of making many LLM calls.

In [ ]:
async def setup_agentic_heuristic_system(client):
    """Instantiate heuristic agents for Scrabble state evaluation."""

    # 1. Define Heuristic Agents (using roles defined in Part 4)
    # The system prompts MUST instruct the agents to output a clear numeric score.
    opponent_status_agent = AssistantAgent(
        name="Opponent_Status_Agent",
        system_message="""
          Determine how well opponent is performing and assign
          a score. Provide a brief explanation and end
          with 'OPPONENT_SCORE: <float>'""",
        model_client=client,
    )

    move_usage_agent = AssistantAgent(
        name="Move_Usage_Agent",
        system_message="""
          Determine how proposed move utilizes the space to
          maximize points earned on this turn. Provide a brief
          explanation and end with'USAGE_SCORE: <float>'""",
        model_client=client,
    )

    future_word_agent = AssistantAgent(
        name="Future_Word_Agent",
        system_message="""Determine how proposed move sets player up for
        more points based on remaining tiles in rack and likelihood of
        forming future words based on game board. Provide a brief explanation
        and end with 'FUTURE_SCORE: <float>'""",
        model_client=client,
    )

    advantage_agent = AssistantAgent(
        name="Advantage_Agent",
        system_message="""
          Gather the information from each of the agents, and determine the
          advantage of each move (as a score). Use a defined weighting
          (for example, 1.5 * Opponent + 2 * Usage + Future * 2.5).

          **Crucially, you must always end your final response with 'FINAL_SCORE: <float>'.**
          Ensure the score is a clear numeric value.
          """,
        model_client=client,
    )

    return [opponent_status_agent, move_usage_agent, future_word_agent], advantage_agent

async def collaborative_heuristic(state: ScrabbleState, player: int) -> float:
    heuristic_agents, aggregator = await setup_agentic_heuristic_system(client)

    task = f"Evaluate the strategic value of this Scrabble State for the current player using collaboration:\n{state}"

    groupchat = RoundRobinGroupChat(
        heuristic_agents + [aggregator],
        max_turns=len(heuristic_agents) + 1, # Allow enough turns for all agents and the aggregator
    )

    # Pass the initial task to the run method
    result = await groupchat.run(task=task)

    # Parse the final output from the aggregator
    final_output = result.messages[-1].content
    score = state.get_utility(state.current_player) # Default to utility if parsing fails
    try:
        # Use regex to find the FINAL_SCORE: <float> pattern
        import re
        match = re.search(r"FINAL_SCORE:\s*(-?\d+(\.\d+)?)", final_output)
        if match:
            score = float(match.group(1))
            print(f"Parsed FINAL_SCORE: {score}") # Add a print for debugging
        else:
            print(f"Warning: Could not find FINAL_SCORE in agent output: {final_output}. Returning baseline utility.")

    except Exception as e:
        print(f"Warning: Failed to parse final score due to error: {e}. Returning baseline utility.")
        print(f"Agent output was: {final_output}")

    return score

# To run the experiment: replace the placeholder function in MCTS Agent
await run_mcts_match(num_playouts=1, heuristic_fn=collaborative_heuristic)

Starting Scrabble game simulation for 5 turns. MCTS Agent (P1) uses 1 playouts.

--- Turn 1: Player 1 (MCTS) ---
Starting MCTS: 1 playouts per move (for 151 possible moves)...
Parsed FINAL_SCORE: 0.577
Parsed FINAL_SCORE: 0.47083
Parsed FINAL_SCORE: 0.5567
Parsed FINAL_SCORE: 0.70417
Parsed FINAL_SCORE: 0.6225
Parsed FINAL_SCORE: 3.05
Parsed FINAL_SCORE: 0.5691666667
Parsed FINAL_SCORE: 0.482
Parsed FINAL_SCORE: 0.656
Parsed FINAL_SCORE: 0.66
Parsed FINAL_SCORE: 0.536
Parsed FINAL_SCORE: 0.66917


---

## Reflection & Analysis

##### What worked well in the Agentic Heuristic System?

The communication was great, and every agent seemed to understand each other.

##### What struggled?

I did have to make the aggregator more explicit because it wasn't outputting a number for FINAL_SCORE. Also, using the agents is incredibly slow and doesn't seem super effective for the problem it's trying to solve.

##### Agentic vs. Classical Heuristics

Agentic systems are slow but seem to have a more nuanced and intuitive response. They can be used for basically any system and with lots of different tools. However, I think it's important to implement agents in an efficient way, maybe broadening the definition of state so that it doesn't have to run so many different prompts (like running the agents on several different moves at once, somehow).